# Propensity analysis

NOTE: What is referred to as a dict is sometimes a df - need to fix this labelling at some point.

## Imports

In [ ]:
import copy
import itertools
import os
import pickle
import random
import scipy
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns  # Note will call statsmodels for kde if installed on system, otherwise will use scipy
from collections import OrderedDict
from sklearn.cluster import KMeans

## Functions and variable definitions

In [ ]:
"""
Defines dictionary of amino acid abbreviations
"""
aa_dict = OrderedDict({'A': 'Ala',
                       'R': 'Arg',
                       'N': 'Asn',
                       'D': 'Asp',
                       'C': 'Cys',
                       'Q': 'Gln',
                       'E': 'Glu',
                       'G': 'Gly', 
                       'H': 'His',
                       'I': 'Ile',
                       'L': 'Leu',
                       'K': 'Lys',
                       'M': 'Met',
                       'F': 'Phe',
                       'P': 'Pro',
                       'S': 'Ser',
                       'T': 'Thr',
                       'W': 'Trp',
                       'Y': 'Tyr',
                       'V': 'Val'})

In [ ]:
#
def define_dict(dict_vals):
    """
    Defines a dictionary of amino acid properties. Allows quick definition of new property dictionaries (can
    simply copy and paste the new property values from e.g. the Amino Acid Index) 
    Input: a list (dict_vals) of property values for the 20 (alphabetically ordered) amino acids
    Returns: a dictionary of these property values
    """
    aa_dict = OrderedDict({'A': dict_vals[0],
                           'R': dict_vals[1],
                           'N': dict_vals[2],
                           'D': dict_vals[3],
                           'C': dict_vals[4],
                           'Q': dict_vals[5],
                           'E': dict_vals[6],
                           'G': dict_vals[7],
                           'H': dict_vals[8],
                           'I': dict_vals[9],
                           'L': dict_vals[10],
                           'K': dict_vals[11],
                           'M': dict_vals[12],
                           'F': dict_vals[13],
                           'P': dict_vals[14],
                           'S': dict_vals[15],
                           'T': dict_vals[16],
                           'W': dict_vals[17],
                           'Y': dict_vals[18],
                           'V': dict_vals[19]})
    return aa_dict

In [ ]:
#
def remove_nan(prop_list):
    """
    Removes all instances of '', 'NaN' and np.nan from an input list
    Input: a list (prop_list) to be filtered
    Returns: the filtered list
    """
    prop_remove_list = ['', 'NaN', 'nan', np.nan]
    for prop_remove in prop_remove_list:
        while prop_remove in prop_list:
            prop_list.remove(prop_remove)
    return prop_list

In [ ]:
#
def calc_distribution(df, aa_dict):
    """
    Counts the number of each amino acid in dataframe of barrel / sandwich properties
    Input: dataframe of barrel / sandwich properties and dictionary of amino acid abbreviations
    Returns: dataframe of count data
    """
    distribution_list = ['']*len(aa_dict.keys())
    fasta_list = [x for x in df['fasta_seq'].tolist() if x in list(aa_dict.keys())]
    for index, aa in enumerate(list(aa_dict.keys())):
        count = fasta_list.count(aa)
        distribution_list[index] = count / len(fasta_list)

    df_dict = OrderedDict({'FASTA': list(aa_dict.keys())})
    df_dict['Normalised frequency'] = distribution_list

    distribution_df = pd.DataFrame(df_dict)
    return distribution_df

In [ ]:
#
def calc_indv_property_propensities(df, prop, aa_dict):
    """
    Calculates propensities of the amino acids listed in the input dictionary for a (categorical) feature of
    interest (e.g. 'interior' or 'exterior')
    Input: dataframe of barrel / sandwich properties, name of the feature of interest (specified via the name of
           the corresponding column in the input dataframe), and dictionary of amino acid abbreviations
    Returns: dataframe of propensity values, plus dataframes of the frequencies and normalised frequencies (since
             extreme propensity values can result from low counts)
    """
    df = df[~df[prop].isin(['', 'NaN', 'nan', np.nan])]
    df = df.reset_index(drop=True)

    prop_list = list(set(df[prop].tolist()))
    prop_list = remove_nan(prop_list)

    temp_lists = {}
    for prop_val in prop_list:
        temp_lists['{}_propensity_list'.format(prop_val)] = ['']*len(list(aa_dict.keys()))
        temp_lists['{}_frequency_list'.format(prop_val)] = ['']*len(list(aa_dict.keys()))
        temp_lists['{}_normed_frequencies_list'.format(prop_val)] = ['']*len(list(aa_dict.keys()))

        temp_lists['total_{}_count'.format(prop_val)] = df[prop].tolist().count(prop_val)

    for index, aa in enumerate(list(aa_dict.keys())):
        aa_df = df[df['fasta_seq'] == aa]
        aa_df = aa_df.reset_index(drop=True)

        for prop_val in prop_list:
            temp_lists['{}_{}_count'.format(prop_val, aa)] = aa_df[prop].tolist().count(prop_val)

            if(
                min([temp_lists['{}_{}_count'.format(prop_val, aa)], temp_lists['total_{}_count'.format(prop_val)],
                     aa_df.shape[0], df.shape[0]]) > 0
            ):
                temp_lists['{}_frequency_list'.format(prop_val)][index] = copy.deepcopy(temp_lists['{}_{}_count'.format(prop_val, aa)])

                temp_lists['{}_{}_normed_frequencies'.format(prop_val, aa)] = (temp_lists['{}_{}_count'.format(prop_val, aa)]
                                                                             / temp_lists['total_{}_count'.format(prop_val)])
                temp_lists['{}_normed_frequencies_list'.format(prop_val)][index] = temp_lists['{}_{}_normed_frequencies'.format(prop_val, aa)]

                temp_lists['{}_{}_propensity'.format(prop_val, aa)] = ((temp_lists['{}_{}_normed_frequencies'.format(prop_val, aa)])
                                                                       / (aa_df.shape[0] / df.shape[0]))
                temp_lists['{}_propensity_list'.format(prop_val)][index] = temp_lists['{}_{}_propensity'.format(prop_val, aa)]
            else:
                temp_lists['{}_frequency_list'.format(prop_val)][index] = np.nan
                temp_lists['{}_normed_frequencies_list'.format(prop_val)][index] = np.nan
                temp_lists['{}_propensity_list'.format(prop_val)][index] = np.nan

    df_propensity_dict = OrderedDict({'FASTA': list(aa_dict.keys())})
    df_frequency_dict = OrderedDict({'FASTA': list(aa_dict.keys())})
    df_normed_frequencies_dict = OrderedDict({'FASTA': list(aa_dict.keys())})
    for prop_val in prop_list:
        df_propensity_dict[prop_val] = temp_lists['{}_propensity_list'.format(prop_val)]
        df_frequency_dict[prop_val] = temp_lists['{}_frequency_list'.format(prop_val)]
        df_normed_frequencies_dict[prop_val] = temp_lists['{}_normed_frequencies_list'.format(prop_val)]

    propensity_df = pd.DataFrame(df_propensity_dict)
    frequency_df = pd.DataFrame(df_frequency_dict)
    normed_frequencies_df = pd.DataFrame(df_normed_frequencies_dict)
    
    return propensity_df, frequency_df, normed_frequencies_df

In [ ]:
#
def calc_combined_property_propensities(df, props, aa_dict):
    """
    Calculates propensities of the amino acids listed in the input dictionary for two or more (categorical)
    features of interest (e.g. 'interior' and 'transmembrane', 'interior' and 'external', 'exterior' and
    'transmembrane', or 'exterior' and 'external')
    Input: dataframe of barrel / sandwich properties, list of names of the features of interest (specified via
           the names of the corresponding columns in the input dataframe), and dictionary of amino acid
           abbreviations
    Returns: dataframe of propensity values, plus dataframes of the frequencies and normalised frequencies (since
             extreme propensity values can result from low counts)
    """
    for prop in props:
        df = df[~df[prop].isin(['', 'NaN', 'nan', np.nan])]
        df = df.reset_index(drop=True)

    amalgamate_list = []
    for prop in props:
        filtered_list = list(set(df[prop].tolist()))
        amalgamate_list.append(filtered_list)
    combinations = list(itertools.product(*amalgamate_list))
    combinations = ['_'.join(tup) for tup in combinations]

    for prop in props:
        df[prop] = [str(x) for x in df[prop].tolist()]
    df_props = pd.DataFrame({'combination': df[props].apply('_'.join, axis=1),
                             'FASTA': df['fasta_seq']})

    temp_lists = {}
    for combination in combinations:
        temp_lists['{}_propensity_list'.format(combination)] = ['']*len(list(aa_dict.keys()))
        temp_lists['{}_frequency_list'.format(combination)] = ['']*len(list(aa_dict.keys()))
        temp_lists['{}_normed_frequencies_list'.format(combination)] = ['']*len(list(aa_dict.keys()))

        temp_lists['total_{}_count'.format(combination)] = df_props['combination'].tolist().count(combination)

        for index, aa in enumerate(list(aa_dict.keys())):
            aa_df = df_props[df_props['FASTA'] == aa]
            aa_df = aa_df.reset_index(drop=True)
            
            temp_lists['{}_{}_count'.format(combination, aa)] = aa_df['combination'].tolist().count(combination)
            if (
                min([temp_lists['{}_{}_count'.format(combination, aa)],
                     temp_lists['total_{}_count'.format(combination)], aa_df.shape[0], df.shape[0]]) > 0
            ):
                temp_lists['{}_frequency_list'.format(combination)][index] = copy.deepcopy(temp_lists['{}_{}_count'.format(combination, aa)])

                temp_lists['{}_{}_normed_frequencies'.format(combination, aa)] = (temp_lists['{}_{}_count'.format(combination, aa)]
                                                                                / temp_lists['total_{}_count'.format(combination)])
                temp_lists['{}_normed_frequencies_list'.format(combination)][index] = temp_lists['{}_{}_normed_frequencies'.format(combination, aa)]

                temp_lists['{}_{}_propensity'.format(combination, aa)] = ((temp_lists['{}_{}_normed_frequencies'.format(combination, aa)])
                                                                          / (aa_df.shape[0] / df_props.shape[0]))
                temp_lists['{}_propensity_list'.format(combination)][index] = temp_lists['{}_{}_propensity'.format(combination, aa)]
            else:
                temp_lists['{}_frequency_list'.format(combination)][index] = np.nan
                temp_lists['{}_normed_frequencies_list'.format(combination)][index] = np.nan
                temp_lists['{}_propensity_list'.format(combination)][index] = np.nan

    df_propensity_dict = OrderedDict({'FASTA': list(aa_dict.keys())})
    df_frequency_dict = OrderedDict({'FASTA': list(aa_dict.keys())})
    df_normed_frequencies_dict = OrderedDict({'FASTA': list(aa_dict.keys())})
    for combination in combinations:
        df_propensity_dict[combination] = temp_lists['{}_propensity_list'.format(combination)]
        df_frequency_dict[combination] = temp_lists['{}_frequency_list'.format(combination)]
        df_normed_frequencies_dict[combination] = temp_lists['{}_normed_frequencies_list'.format(combination)]

    propensity_df = pd.DataFrame(df_propensity_dict)
    frequency_df = pd.DataFrame(df_frequency_dict)
    normed_frequencies_df = pd.DataFrame(df_normed_frequencies_dict)

    return propensity_df, frequency_df, normed_frequencies_df

In [ ]:
#
def gen_neighbouring_pairs_list(df, interaction, aa_dict, *args):
    """
    Makes a list of the amino acids in the input dataframe that form pairwise interactions at a particular
    location (e.g. HB vs. NHB positions) or via a particular type of interaction (e.g. cation-pi, hydrogen
    bonds, van der Waals interactions, etc.)
    Input: dataframe of barrel / sandwich properties, name of the interaction type / location of interest
           (specified via the name of the corresponding column in the input dataframe), dictionary of amino
           acid abbreviations, and list of properties against which to measure the amino acid pairs (e.g. phi
           and psi)
    Returns: list of interacting amino acid pairs, and (if args) dictionaries of args values for amino acid
             pairs
    """
    if args:
        props = list(args)

    # Makes list of amino acid pairs (of which both amino acids are in the dataframe)
    domain_res_ids_list = []
    for row in range(df.shape[0]):
        domain_res_id = df['domain_ids'][row] + df['res_ids'][row]
        domain_res_ids_list.append(domain_res_id)

    neighbouring_pairs_list = []
    repeat_pairs_list = []
    if args:
        props_dict = OrderedDict()
        for prop in props:
            props_dict[prop] = OrderedDict()

    for row in range(df.shape[0]):
        domain_res_id_1 = df['domain_ids'][row] + df['res_ids'][row]
        aa_1 = df['fasta_seq'][row]
        if args:
            prop_val_1_list = ['']*len(props)
            for index, prop in enumerate(props):
                prop_val_1_list[index] = df[prop][row]

        interaction_list = df[interaction][row]
        if type(interaction_list) != list:
            interaction_list = [interaction_list]
        for res_id_2 in interaction_list:
            domain_res_id_2 = df['domain_ids'][row] + res_id_2

            # Only considers amino acid pairs in which both (canonical) amino acids are located in the domain
            if domain_res_id_2 in domain_res_ids_list:
                res_id_2_index = domain_res_ids_list.index(domain_res_id_2)
                aa_2 = df['fasta_seq'][res_id_2_index]
                if args:
                    prop_val_2_list = ['']*len(props)
                    for index, prop in enumerate(props):
                        prop_val_2_list[index] = df[prop][res_id_2_index]

                if (
                        aa_1 in list(aa_dict.keys())
                    and aa_2 in list(aa_dict.keys())
                    and (not [domain_res_id_1, domain_res_id_2] in repeat_pairs_list)
                    and (not [domain_res_id_2, domain_res_id_1] in repeat_pairs_list)
                ):  # Each amino acid pair is counted once.
                    repeat_pairs_list.append([domain_res_id_1, domain_res_id_2])
                    repeat_pairs_list.append([domain_res_id_2, domain_res_id_1])

                    neighbouring_pairs_list.append('{}_{}'.format(aa_1, aa_2))
                    
                    if args:
                        for index, prop in enumerate(props):
                            prop_val_1 = prop_val_1_list[index]
                            prop_val_2 = prop_val_2_list[index]
                            # Must include domain id in key to ensure it is unique
                            props_dict[prop]['{}_{}_{}_{}'.format(domain_res_id_1, domain_res_id_2, aa_1, aa_2)] = [
                                prop_val_1, prop_val_2
                            ]
                            props_dict[prop]['{}_{}_{}_{}'.format(domain_res_id_2, domain_res_id_1, aa_2, aa_1)] = [
                                prop_val_2, prop_val_1
                            ]
    if args:
        return neighbouring_pairs_list, props_dict
    else:
        return neighbouring_pairs_list


#
def calc_aa_pair_propensities(neighbouring_pairs_list, aa_dict):
    """
    Calculates propensities of amino acids at a particular location to interact with one another
    Input: list of interacting amino acid pairs output from gen_neighbouring_pairs_list, and dictionary of amino
           acid abbreviations
    Returns: dataframe of propensity values, plus dataframes of the frequencies and normalised frequencies (since
             extreme propensity values can result from low counts)
    """
    # Each amino acid pair is counted in both orientations. (Note this is because positions 1 and 2 must be
    # treated independently (position 1 = object, position 2 = property of object) in the propensity calculations,
    # whereas these pairs do not have such an associated order and so must be listed in both orientations. See
    # lab notes for further details.)
    neighbouring_pairs_list = [['{}_{}'.format(aa_pair[0:1], aa_pair[-1:]), '{}_{}'.format(aa_pair[-1:], aa_pair[0:1])]
                               for aa_pair in neighbouring_pairs_list]
    neighbouring_pairs_list = [aa_pair for aa_pair_list in neighbouring_pairs_list for aa_pair in aa_pair_list]

    # Calculates propensity values
    all_pairs_count = len(neighbouring_pairs_list)
    temp_lists = {}

    for aa_1 in list(aa_dict.keys()):
        temp_lists['{}_propensity_list'.format(aa_1)] = ['']*len(list(aa_dict.keys()))
        temp_lists['{}_frequency_list'.format(aa_1)] = ['']*len(list(aa_dict.keys()))
        temp_lists['{}_normed_frequencies_list'.format(aa_1)] = ['']*len(list(aa_dict.keys()))

        # Calculates propensity for aa_1 to interact with aa_2 relative to all other amino acid possibilities
        # (= (aa_1_aa_2_count / aa_1_count) / (aa_2_count / all_aa_count)). 
        for aa_2_index, aa_2 in enumerate(list(aa_dict.keys())):
            aa_1_count = 0
            aa_2_count = 0
            aa_1_aa_2_count = 0

            for aa_pair in neighbouring_pairs_list:
                if aa_1 == aa_pair[0:1]:
                    aa_1_count += 1
                if aa_2 == aa_pair[-1:]:
                    aa_2_count += 1
                if aa_1 == aa_pair[0:1] and aa_2 == aa_pair[-1:]:
                    aa_1_aa_2_count += 1

            if min([all_pairs_count, aa_1_count, aa_2_count, aa_1_aa_2_count]) > 0:
                aa_1_aa_2_frequency = copy.deepcopy(aa_1_aa_2_count)
                temp_lists['{}_frequency_list'.format(aa_1)][aa_2_index] = aa_1_aa_2_frequency

                aa_1_aa_2_normed_frequencies = aa_1_aa_2_count / aa_1_count
                temp_lists['{}_normed_frequencies_list'.format(aa_1)][aa_2_index] = aa_1_aa_2_normed_frequencies

                aa_1_aa_2_propensity = ((aa_1_aa_2_count / aa_1_count)
                                        / (aa_2_count / all_pairs_count))
                temp_lists['{}_propensity_list'.format(aa_1)][aa_2_index] = aa_1_aa_2_propensity
            else:
                temp_lists['{}_frequency_list'.format(aa_1)][aa_2_index] = np.nan
                temp_lists['{}_normed_frequencies_list'.format(aa_1)][aa_2_index] = np.nan
                temp_lists['{}_propensity_list'.format(aa_1)][aa_2_index] = np.nan

    # The propensity for aa_1 to interact with aa_2 should equal the propensity for aa_2 to interact with aa_1
    for aa_1_index, aa_1 in enumerate(list(aa_dict.keys())):
        for aa_2_index, aa_2 in enumerate(list(aa_dict.keys())):
            propensity_12 = temp_lists['{}_propensity_list'.format(aa_1)][aa_2_index]
            propensity_21 = temp_lists['{}_propensity_list'.format(aa_2)][aa_1_index]

            if not np.isnan(propensity_12) and not np.isnan(propensity_21):
                if propensity_12 - propensity_21 > 0.0001:
                    sys.exit('Error with propensity calculation: {}{} ({}) != {}{} ({})'.format(
                        aa_1, aa_2, propensity_12, aa_2, aa_1, propensity_21
                    ))

    df_propensity_dict = OrderedDict({'FASTA': list(aa_dict.keys())})
    df_frequency_dict = OrderedDict({'FASTA': list(aa_dict.keys())})
    df_normed_frequencies_dict = OrderedDict({'FASTA': list(aa_dict.keys())})
    # FASTA label on heatmap refers to aa_2 (although propensity and frequency heatmaps will be symmetric about
    # their central axes)
    for aa_1 in list(aa_dict.keys()):
        df_propensity_dict[aa_1] = temp_lists['{}_propensity_list'.format(aa_1)]
        df_frequency_dict[aa_1] = temp_lists['{}_frequency_list'.format(aa_1)]
        df_normed_frequencies_dict[aa_1] = temp_lists['{}_normed_frequencies_list'.format(aa_1)]

    propensity_df = pd.DataFrame(df_propensity_dict)
    frequency_df = pd.DataFrame(df_frequency_dict)
    normed_frequencies_df = pd.DataFrame(df_normed_frequencies_dict)

    return propensity_df, frequency_df, normed_frequencies_df

In [ ]:
#
def calc_voronoi_points(df, cont_prop_1, cont_prop_2, min_cluster_size=25, save_file=False, file_name=''):
    """
    Calculates cluster centres for discretisation of 2D property space.
    Input: dataframe of barrel / sandwich properties, first (continuous) property of interest (specified via the
           name of the correspoing column in the input dataframe), second (continuous) property of interest
           (specified via the name of the correspoing column in the input dataframe), the minimum number of
           data points to be included in a cluster (default 25), plus whether to save a copy of the voronoi point
           plot and if so what file name to save it under
    Returns: numpy array of cluster coordinates, plus dataframe of cluster properties
    """
    df = df[  (~df[cont_prop_1].isin(['', 'NaN', 'nan', np.nan]))
            & (~df[cont_prop_2].isin(['', 'NaN', 'nan', np.nan]))]
    df = df.reset_index(drop=True)

    prop_array = []
    for row in range(df.shape[0]):
        prop_array.append([df[cont_prop_1][row], df[cont_prop_2][row]])
    prop_array = np.array(prop_array)

    kmeans = KMeans(n_clusters=200).fit(prop_array)

    if kmeans.n_iter_ >= 300:
        sys.exit('Clustering failed to converge')

    else:
        cluster_coords = kmeans.cluster_centers_

        cluster_sizes = np.unique(kmeans.labels_, return_counts=True)
        cluster_sizes = cluster_sizes[1]

        smallest_cluster_size = cluster_sizes.min()
        while smallest_cluster_size < min_cluster_size:
            smallest_cluster_index = np.abs(cluster_sizes).argmin()
            smallest_cluster_coords = cluster_coords[smallest_cluster_index]

            distances = np.sqrt(np.sum(np.square(cluster_coords-smallest_cluster_coords), axis=1))
            distances[smallest_cluster_index] = 1e10
            closest_cluster_index = np.abs(distances).argmin()
            closest_cluster_size = cluster_sizes[closest_cluster_index]

            cluster_sizes[closest_cluster_index] = closest_cluster_size + smallest_cluster_size
            cluster_sizes = np.delete(cluster_sizes, smallest_cluster_index)
            cluster_coords = np.delete(cluster_coords, smallest_cluster_index, axis=0)

            smallest_cluster_size = cluster_sizes.min()

        plt.clf()
        plt.figure(figsize=(15, 15))
        plt.scatter(prop_array[:,0], prop_array[:,1], alpha=0.1)
        plt.scatter(cluster_coords[:,0], cluster_coords[:,1], color='k')
        if save_file is True:
            plt.savefig('{}.svg'.format(file_name))
        plt.show()

        cluster_df = pd.DataFrame({'Cluster size': cluster_sizes,
                                   '{}'.format(cont_prop_1): cluster_coords[:,0],
                                   '{}'.format(cont_prop_2): cluster_coords[:,1]})

        return cluster_coords, cluster_df


#
def calc_discrete_2d_indv_aa_propensities(df, cont_prop_1, cont_prop_2, aa_dict, cluster_coords):
    """
    Calculates amino acid propensity values for bins in 2D property space.
    Input: dataframe of barrel / sandwich properties, first (continuous) property of interest (specified via the
           name of the corresponding column in the input dataframe), second (continuous) property of interest
           (specified via the name of the corresponding column in the input dataframe), dictionary of amino acid
           abbreviations, and coordinates of cluster centres
    Returns: dataframe of propensity values, plus dataframes of frequency and normalised frequency values (since
             propensity values can be skewed by very small sample sizes)
    """
    df = df[  (~df[cont_prop_1].isin(['', 'NaN', 'nan', np.nan]))
            & (~df[cont_prop_2].isin(['', 'NaN', 'nan', np.nan]))]
    df = df.reset_index(drop=True)

    propensity_dict = OrderedDict({'FASTA': list(aa_dict.keys())})
    frequency_dict = OrderedDict({'FASTA': list(aa_dict.keys())})
    normed_frequencies_dict = OrderedDict({'FASTA': list(aa_dict.keys())})
    voronoi_class_dict = OrderedDict()

    for voronoi_index in range(cluster_coords.shape[0]):
        voronoi_class_dict[voronoi_index] = [0]*(len(aa_dict)+1)
        propensity_dict[voronoi_index] = [np.nan]*len(aa_dict)
        frequency_dict[voronoi_index] = [np.nan]*len(aa_dict)
        normed_frequencies_dict[voronoi_index] = [np.nan]*len(aa_dict)

    for row in range(df.shape[0]):
        fasta = df['fasta_seq'][row]
        aa_index = list(aa_dict.keys()).index(fasta)

        val_1 = df[cont_prop_1][row]
        val_2 = df[cont_prop_2][row]
        prop_vals = np.array([val_1, val_2])

        distances = np.sqrt(np.sum(np.square(cluster_coords-prop_vals), axis=1))
        voronoi_index = np.abs(distances).argmin()
        voronoi_class_dict[voronoi_index][aa_index] += 1
        voronoi_class_dict[voronoi_index][-1] +=1

    for voronoi_index in list(voronoi_class_dict.keys()):
        for aa_index, aa in enumerate(list(aa_dict.keys())):
            class_aa_count = voronoi_class_dict[voronoi_index][aa_index]
            class_all_aas_count = voronoi_class_dict[voronoi_index][-1]

            total_aa_count = 0
            for voronoi_class_list in list(voronoi_class_dict.values()):
                total_aa_count += voronoi_class_list[aa_index]

            try:
                frequency_dict[voronoi_index][aa_index] = copy.deepcopy(class_aa_count)
                normed_frequencies_dict[voronoi_index][aa_index] = class_aa_count / class_all_aas_count
                propensity_dict[voronoi_index][aa_index] = (  (class_aa_count / class_all_aas_count)
                                                            / (total_aa_count / df.shape[0]))
            except ZeroDivisionError:
                pass

    propensity_df = pd.DataFrame(propensity_dict)
    frequency_df = pd.DataFrame(frequency_dict)
    normed_frequencies_df = pd.DataFrame(normed_frequencies_dict)

    return propensity_df, frequency_df, normed_frequencies_df

In [ ]:
#
def bootstrap_discrete_propensities(df, prop, aa_dict, num_bootstrap_samples, propensity_calc_func,
                                    cluster_coords=[]):
    """
    Calculates 95% confidence limits about propensity values calculated for discrete categorical features
    Input: dataframe of barrel / sandwich properties, name(s) of the feature(s) of interest (specified via a
           name / list of names of the corresponding column(s) in the input dataframe, dictionary of amino
           acid abbreviations, the number of bootstrap samples to collect, the name of the propensity
           calculation function (calc_indv_property_propensities, calc_combined_property_propensities,
           calc_aa_pair_propensities, or calc_discrete_2d_indv_aa_propensities), and the coordinates of each
           cluster centre (if calculating propensities for bins in 2D property space, e.g. phi vs. psi)
    Returns: dictionary of bootstrapped propensity values, plus dictionaries of the bootstrapped frequencies
             and normalised frequencies (since extreme propensity values can result from low counts)
    """
    if propensity_calc_func == calc_indv_property_propensities:
        prop_vals = list(set(df[prop].tolist()))
        prop_vals = remove_nan(prop_vals)

    elif propensity_calc_func == calc_combined_property_propensities:
        prop_vals = []
        for sub_prop in prop:
            vals = list(set(df[sub_prop].tolist()))
            vals = remove_nan(vals)
            prop_vals.append(vals)
        prop_vals = list(itertools.product(*prop_vals))
        prop_vals = ['_'.join(tup) for tup in prop_vals]

    elif propensity_calc_func == calc_aa_pair_propensities:
        prop_vals = list(aa_dict.keys())
        neighbouring_pairs_list = gen_neighbouring_pairs_list(df, prop, aa_dict)

    elif propensity_calc_func == calc_discrete_2d_indv_aa_propensities:
        prop_vals = [num for num in range(cluster_coords.shape[0])]

    aa_list = list(aa_dict.keys())

    bootstrap_propensities_dict = OrderedDict({'FASTA': aa_list})
    bootstrap_frequencies_dict = OrderedDict({'FASTA': aa_list})
    bootstrap_normed_frequencies_dict = OrderedDict({'FASTA': aa_list})

    for prop_val in prop_vals:
        bootstrap_propensities_dict[prop_val] = [([np.nan]*num_bootstrap_samples) for i in range(len(aa_list))]
        bootstrap_frequencies_dict[prop_val] = [([np.nan]*num_bootstrap_samples) for i in range(len(aa_list))]
        bootstrap_normed_frequencies_dict[prop_val] = [([np.nan]*num_bootstrap_samples) for i in range(len(aa_list))]

    for num_1 in range(num_bootstrap_samples):
        if propensity_calc_func in [calc_indv_property_propensities, calc_combined_property_propensities,
                                    calc_discrete_2d_indv_aa_propensities]:
            # Samples input dataframe with replacement
            indices = []
            for num_2 in range(df.shape[0]):
                random_num = random.randint(0, (df.shape[0]-1))
                indices.append(random_num)
            new_df = copy.deepcopy(df).iloc[indices]
            new_df = new_df.reset_index(drop=True)

            if propensity_calc_func in [calc_indv_property_propensities, calc_combined_property_propensities]:
                propensity_df, frequency_df, normed_frequencies_df = propensity_calc_func(
                    new_df, prop, aa_dict
                )
            elif propensity_calc_func == calc_discrete_2d_indv_aa_propensities:
                propensity_df, frequency_df, normed_frequencies_df = propensity_calc_func(
                    new_df, prop[0], prop[1], aa_dict, cluster_coords
                )

        elif propensity_calc_func == calc_aa_pair_propensities:
            # Samples neighbouring_pairs_list with replacement
            resampled_pairs_list = random.choices(neighbouring_pairs_list, k=len(neighbouring_pairs_list))
            
            propensity_df, frequency_df, normed_frequencies_df = propensity_calc_func(
                resampled_pairs_list, aa_dict
            )

        for prop_val in prop_vals:
            for index, num_3 in enumerate(propensity_df[prop_val].tolist()):
                bootstrap_propensities_dict[prop_val][index][num_1] = propensity_df[prop_val][index]
                bootstrap_frequencies_dict[prop_val][index][num_1] = frequency_df[prop_val][index]
                bootstrap_normed_frequencies_dict[prop_val][index][num_1] = normed_frequencies_df[prop_val][index]

    return bootstrap_propensities_dict, bootstrap_frequencies_dict, bootstrap_normed_frequencies_dict

In [ ]:
#
def plot_bar_graphs(df, propensity_or_frequency, save_file=True, file_name=''):
    """
    Plots a bar graph of property / propensity of interest vs. amino acid identity
    Input: dataframe of barrel / sandwich properties, and string defining whether the input dataframe contains
           propensity, frequency or normalised frequency values, whether to save a copy of the plot and if so
           the file name to use
           (specified via the name of the corresponding column in the input dataframe)
    """
    reshaped_df = pd.melt(df, id_vars='FASTA', var_name='Property', value_name=propensity_or_frequency)
    reshaped_df = reshaped_df.dropna()
    reshaped_df = reshaped_df.reset_index(drop=True)

    plt.clf()
    sns.catplot(
        x='FASTA', y=propensity_or_frequency, hue='Property', data=reshaped_df, kind='bar', height=5, aspect=3,
        legend=False
    )
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    if save_file is True:
        plt.savefig('{}.svg'.format(file_name))
    plt.show()

In [ ]:
#
def gen_95_conf_intervals(bootstrap_propensity_dict, propensity_or_frequency, middle=False):
    """
    Calculates 95% confidence intervals from bootstrapped sample.
    Input: dictionary of bootstrapped propensity values, and string defining whether the input dataframe contains
           propensity, frequency or normalised frequency values
    Returns: dictionary of lower percentile propensity values, dictionary of upper percentile propensity values
    """
    aa_list = bootstrap_propensity_dict['FASTA']
    
    lower_percentile_dict = OrderedDict({'FASTA': aa_list})
    upper_percentile_dict = OrderedDict({'FASTA': aa_list})
    if middle is True:
        middle_percentile_dict = OrderedDict({'FASTA': aa_list})

    for prop_val in list(bootstrap_propensity_dict.keys()):
        if prop_val != 'FASTA':
            lower_percentile_dict[prop_val] = [np.nan]*len(aa_list)
            upper_percentile_dict[prop_val] = [np.nan]*len(aa_list)
            if middle is True:
                middle_percentile_dict[prop_val] = [np.nan]*len(aa_list)

            for index, propensity_list in enumerate(bootstrap_propensity_dict[prop_val]):
                propensity_list = [val for val in propensity_list if not np.isnan(val)]
                if propensity_list:
                    lower_percentile = np.percentile(propensity_list, 2.5)
                    upper_percentile = np.percentile(propensity_list, 97.5) 
                    middle_percentile = np.percentile(propensity_list, 50)
                else:
                    lower_percentile = np.nan
                    upper_percentile = np.nan
                    middle_percentile = np.nan

                lower_percentile_dict[prop_val][index] = lower_percentile
                upper_percentile_dict[prop_val][index] = upper_percentile
                if middle is True:
                    middle_percentile_dict[prop_val][index] = middle_percentile

    if middle is False:
        return lower_percentile_dict, upper_percentile_dict
    if middle is True:
        return lower_percentile_dict, upper_percentile_dict, middle_percentile_dict


#
def plot_bar_graphs_with_conf_limits(bootstrap_df, lower_percentile_dict, upper_percentile_dict,
                                     propensity_or_frequency, save_file=False, file_name=''):
    """
    Plots a bar graph of property / propensity of interest vs. amino acid identity with bootstrapped 95%
    confidence limits
    Input: dataframe of initial propensity values, dataframe of bootstrapped lower percentile values, dataframe
           of bootstrapped upper percentile values, and string defining whether the input dataframe contains
           propensity, frequency or normalised frequency values, whether to save the plot and if so the file
           name to use
    """
    reshaped_df = pd.melt(bootstrap_df, id_vars='FASTA', var_name='Property', value_name=propensity_or_frequency)
    na_indices = list(np.where(reshaped_df[propensity_or_frequency].notnull())[0])
    reshaped_df = reshaped_df.iloc[na_indices].reset_index(drop=True)

    lower_percentile_df = pd.DataFrame(lower_percentile_dict)
    lower_percentile_df = pd.melt(lower_percentile_df, id_vars='FASTA', var_name='Property',
                                  value_name=propensity_or_frequency)
    lower_percentile_df = lower_percentile_df.iloc[na_indices].reset_index(drop=True)

    upper_percentile_df = pd.DataFrame(upper_percentile_dict)
    upper_percentile_df = pd.melt(upper_percentile_df, id_vars='FASTA', var_name='Property',
                                  value_name=propensity_or_frequency)
    upper_percentile_df = upper_percentile_df.iloc[na_indices].reset_index(drop=True)

    # Plots bar graph with confidence limits
    num_prop = len(set(reshaped_df['Property'].tolist()))
    if num_prop <= 10:
        palette = 'dark'
    else:
        palette = ['black']
    plt.clf()
    plt.figure(figsize=(15,8))
    sns.barplot(x='FASTA', y=propensity_or_frequency, hue='Property', data=reshaped_df)
    sns.stripplot(x='FASTA', y=propensity_or_frequency, hue='Property', data=lower_percentile_df,
                  jitter=False, dodge=True, palette=palette)
    sns.stripplot(x='FASTA', y=propensity_or_frequency, hue='Property', data=upper_percentile_df,
                  jitter=False, dodge=True, palette=palette)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    if save_file is True:
        plt.savefig('{}.svg'.format(file_name))
    plt.show()

In [ ]:
#
def print_conf_limits(prop_list, aa_list, lower_percentile_dict, df, upper_percentile_dict):
    """
    Prints confidence limits about discrete property propensity values for each aa
    Input: list of discrete properties, list of amino acids, dictionary of lower percentile values, dataframe of
           propensity (or frequency or normalised frequency) values, dictionary of upper percentile values
    """
    for prop in prop_list:
        for index, aa in enumerate(aa_list):
            print(aa, prop)
            print('[{}, {}, {}]'.format(
                lower_percentile_dict[prop][index], df[prop][index], upper_percentile_dict[prop][index]
            ))

In [ ]:
#
def iterate_bootstrap_propensities(df, prop, aa_dict, bootstrap_samples_list, propensity_calc_func, propensity_df,
                                   frequency_df, cluster_coords=[]):
    """
    Calculates 95% confidence limits about propensity, frequency and normalised frequency values calculated for
    discrete categorical features whilst varying the size of the bootstrap population (to determine the minimum
    number of bootstrap samples required to obtain an accurate estimate of the confidence limits)
    Input: dataframe of barrel / sandwich properties, name(s) of the feature(s) of interest (specified via a
           name / list of names of the corresponding column(s) in the input dataframe), dictionary of amino
           acid abbreviations, list of number of bootstrap samples to use in the confidence limit calculation,
           name of the propensity calculation function (calc_indv_property_propensities,
           calc_combined_property_propensities, calc_aa_pair_propensities, or calc_discrete_2d_indv_aa_propensities),
           dataframe of (non-bootstrapped) propensity values, dataframe of (non-bootstrapped) frequency values,
           dataframe of (non-bootstrapped) normalised frequency values, and the coordinates of each cluster centre
           (if calculating propensities for bins in 2D property space, e.g. phi vs. psi)
    Returns: dictionaries of propensity, frequency and normalised frequency values vs. number of bootstrap samples
    """
    dfs = {'Propensity': propensity_df,
           'Frequency': frequency_df}

    iterated_dicts = {'Propensity': OrderedDict(),
                      'Frequency': OrderedDict()}

    for num in bootstrap_samples_list:
        print(num)
        (bootstrap_propensities_dict, bootstrap_frequencies_dict, bootstrap_normed_frequencies_dict
        ) = bootstrap_discrete_propensities(df, prop, aa_dict, num, propensity_calc_func, cluster_coords)
        bootstrapped_dicts = {'Propensity': bootstrap_propensities_dict,
                              'Frequency': bootstrap_frequencies_dict}

        for output_type, iterated_dict in iterated_dicts.items():
            bootstrap_dict = bootstrapped_dicts[output_type]
            
            if propensity_calc_func in [calc_indv_property_propensities, calc_combined_property_propensities,
                                        calc_aa_pair_propensities]:
                lower_percentile, upper_percentile = gen_95_conf_intervals(
                    bootstrap_dict, output_type
                )
            elif propensity_calc_func == calc_discrete_2d_indv_aa_propensities:
                lower_percentile, upper_percentile, middle_percentile = gen_95_conf_intervals(
                    bootstrap_dict, output_type, middle=True
                )
            iterated_dict[num] = OrderedDict({'FASTA': list(aa_dict.keys())})

            for prop_val in list(bootstrap_dict.keys()):
                if prop_val != 'FASTA':
                    iterated_dict[num][prop_val] = ['']*len(list(aa_dict.keys()))

                    if propensity_calc_func in [calc_indv_property_propensities, calc_combined_property_propensities,
                                                calc_aa_pair_propensities]:
                        for index, aa in enumerate(list(aa_dict.keys())):
                            iterated_dict[num][prop_val][index] = [
                                lower_percentile[prop_val][index], dfs[output_type][prop_val][index],
                                upper_percentile[prop_val][index]
                            ]
                    elif propensity_calc_func == calc_discrete_2d_indv_aa_propensities:
                        for index, aa in enumerate(list(aa_dict.keys())):
                            iterated_dict[num][prop_val][index] = [
                                lower_percentile[prop_val][index], middle_percentile[prop_val][index],
                                upper_percentile[prop_val][index]
                            ]

    for output_type, iterated_dict in iterated_dicts.items():
        for index, aa in enumerate(list(aa_dict.keys())):
            print(aa)
            x_vals = []
            y_vals = []
            hue_vals = []

            for prop_val in list(iterated_dict[min(bootstrap_samples_list)].keys()):
                if prop_val != 'FASTA':
                    for num in bootstrap_samples_list:
                        percentiles = iterated_dict[num][prop_val][index]
                        x_vals += [num]*3
                        y_vals += [percentiles[0], percentiles[1], percentiles[2]]
                        hue_vals += [prop_val]*3
            
            melted_df = pd.DataFrame({'Bootstrap samples': x_vals,
                                      '{}'.format(output_type): y_vals,
                                      'Property': hue_vals})
            plt.clf()
            plt.figure(figsize=(8, 5))
            sns.swarmplot(x='Bootstrap samples', y=output_type, hue='Property', data=melted_df)
            plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
            plt.show()

    return iterated_dicts['Propensity'], iterated_dicts['Frequency']

In [ ]:
#
def plot_heat_map(df, save_file=False, file_name=''):
    """
    Plots a heat map.
    Input: dataframe of propensity / count data of property of interest vs. amino acid identity (which must be in
           a column labelled 'FASTA')
    """
    df = df.set_index('FASTA', drop=True)  # "FASTA" label on y-axis of heat map refers to amino acid 2

    plt.clf()
    plt.figure(figsize=(20, 10))
    sns.heatmap(df, cmap='coolwarm')
    if save_file is True:
        plt.savefig('{}.svg'.format(file_name))
    plt.show()
        

#
def plot_ramachandran_heat_map(propensity_df, cluster_coords, save_file=False, file_name=''):
    """
    For each amino acid, generates scatter plot of the cluster centres of the phi/psi bins previously
    calculated with calc_voronoi_points, coloured by the propensity/freqency of the amino acid for that bin.
    Input: DataFrame of propensity/frequency values for each of the cluster centres, numpy array of the cluster
           centre coordinates, whether to save a copy of the plots and if so the file names to use
    """
    for r in range(propensity_df.shape[0]):
        aa = propensity_df['FASTA'][r]
        print(aa)

        x = list(cluster_coords[:,0])
        y = list(cluster_coords[:,1])
        propensity = propensity_df.iloc[r,1:].tolist()

        data = pd.DataFrame({'phi': x, 'psi': y, 'propensity': propensity})
        plt.clf()
        fig = sns.relplot(
            data=data, x='phi', y='psi', hue='propensity', sizes=(10, 200), height=7, aspect=1.2,
            palette='coolwarm'
        )
        if save_file is True:
            plt.savefig('{}_{}.svg'.format(aa, file_name))
        plt.show()

In [ ]:
#
def plot_aa_kdes(df, cont_props, indv_or_comp, aa_dict, save_file=False, file_name=''):
    """
    Plots kernel density estimate of the distribution of each amino acid of interest vs. 1 / 2 continuous
    property/ies of interest (typically z-coordinate)
    Input: dataframe of barrel / sandwich properties, name of continuous property of interest (specified via a list
           of the name(s) of the corresponding column in the input dataframe), whether to plot the distribution of
           the amino acid alone ('individual') or whether to also plot the distribution of all amino acids
           ('comparison'), dictionary of amino acid abbreviations, whether to save a copy of each plot and if so the
           base file name to save each under
    """
    for prop in cont_props:
        df = df[~df[prop].isin(['', 'NaN', 'nan', np.nan])]
        df = df.reset_index(drop=True)

    for aa in list(aa_dict.keys()):
        aa_df = df[df['fasta_seq'] == aa]
        aa_df = aa_df.reset_index(drop=True)

        num_prop_vals = []
        for prop in cont_props:
            num_prop_vals.append(len(df[prop]))
            num_prop_vals.append(len(aa_df[prop]))

        if min(num_prop_vals) >= 10:
            print(aa)

            plt.clf()
            plt.figure(figsize=(10, 5))
            if 'z_coords' in cont_props:
                axes = plt.gca()

            if len(cont_props) == 1:
                if indv_or_comp == 'comparison':
                    sns.distplot(df[cont_props[0]], hist=False, kde_kws={'bw':'scott'})
                sns.distplot(aa_df[cont_props[0]], hist=False, rug=True, kde_kws={'bw':'scott'},
                             rug_kws={'alpha': 0.1})
            elif len(cont_props) == 2:
                if indv_or_comp == 'comparison':
                    sns.kdeplot(data=df[cont_props[0]], data2=df[cont_props[1]], bw='scott', shade=True,
                                shade_lowest=False, cbar=True)
                sns.kdeplot(data=aa_df[cont_props[0]], data2=aa_df[cont_props[1]], bw='scott', shade=False, cbar=True)
            else:
                return('Too many properties specified')

            if save_file is True:
                plt.savefig('{}_{}_{}.svg'.format(aa, '_'.join([prop for prop in cont_props]), file_name))
            plt.show()
            
            
#
def plot_bootstrap_aa_kdes(df, cont_prop, aa_dict, bootstrap_num, save_file=False, file_name=''):
    """
    Generates boostrap_num bootstraps of amino acid df and plots kde of each bootstrap, to give an approximate
    idea of the confidence intervals about the kde of the non-bootstrapped amino acid df 
    Input: dataframe of barrel / sandwich properties, name of continuous property of interest, dictionary of
           amino acid abbreviations, the number of bootstraps to calculate, whether to save a copy of the plot
           and if so the file name to save it under
    """
    
    for aa in list(aa_dict.keys()):
        print(aa)
        aa_df = df[df['fasta_seq'] == aa]
        aa_df = aa_df.reset_index(drop=True)

        plt.clf()
        plt.figure(figsize=(10, 5))
        for num_1 in range(bootstrap_num):
            indices = []
            for num_2 in range(aa_df.shape[0]):
                random_num = random.randint(0, (aa_df.shape[0]-1))
                indices.append(random_num)
            new_df = copy.deepcopy(aa_df).iloc[indices]
            new_df = new_df.reset_index(drop=True)

            sns.kdeplot(data=new_df[cont_prop], bw='scott', color='tab:blue', legend=False)
        sns.kdeplot(data=new_df[cont_prop], bw='scott', color='k', legend=False)

        if save_file is True:
            plt.savefig('{}_{}.svg'.format(aa, file_name))
        plt.show()


In [ ]:
# See notebook prior to 21.02.2021 for equivalent 2D code (for now deleted as insufficient data to run 2D function)
def plot_1d_indv_aa_propensities(df, cont_prop, aa_dict, save_file=False, file_name=True):
    """
    Plots propensity of each amino acid of interest vs. a continuous property of interest (typically z-coordinate)
    Input: dataframe of barrel / sandwich properties, name of continuous property of interest (specified via the
           name of the corresponding column in the input dataframe), and dictionary of amino acid abbreviations,
           whether to save a copy of each plot and if so the base file name to save them under
    """
    df = df[~df[cont_prop].isin(['', 'NaN', 'nan', np.nan])]
    df = df.reset_index(drop=True)

    propensities_dict = OrderedDict()

    for aa in list(aa_dict.keys()):
        aa_df = df[df['fasta_seq'] == aa]
        aa_df = aa_df.reset_index(drop=True)

        if (
            min([df.shape[0], aa_df.shape[0]]) >= 25  # Threshold is a self-determined "sensible" value - is there a way of determining this threshold empirically?
        ):
            print(aa)

            plt.clf()
            overall_data = np.asarray(df[cont_prop].tolist())
            overall_data = overall_data.astype(np.float64)
            x_values_overall, y_values_overall = sns.distributions._statsmodels_univariate_kde(
                data=overall_data, kernel='gau', bw='scott', gridsize=100, cut=3, clip=(-np.inf, np.inf),
                cumulative=False
            )

            # Determines range over which interpolation is carried out
            min_x_vals = [min(df[cont_prop]), min(aa_df[cont_prop])]
            max_x_vals = [max(df[cont_prop]), max(aa_df[cont_prop])]
            x_range = [max(min_x_vals), min(max_x_vals)]

            plt.clf()
            indv_data = np.asarray(aa_df[cont_prop].tolist())
            indv_data = indv_data.astype(np.float64)
            x_values_indv, y_values_indv = sns.distributions._statsmodels_univariate_kde(
                data=indv_data, kernel='gau', bw='scott', gridsize=100, cut=3, clip=(-np.inf, np.inf),
                cumulative=False
            )

            x_values_indv_copy = list(copy.deepcopy(x_values_indv))
            propensities = ['']*len(x_values_indv)

            # Interpolate overall KDE since it has been calculated from more data points
            for index_1, value_1 in np.ndenumerate(x_values_indv):
                index_1 = index_1[0]
                x_indv = x_values_indv[index_1]
                y_indv = y_values_indv[index_1]

                if (   x_indv < x_range[0]
                    or x_indv > x_range[-1]
                ):
                    x_values_indv_copy[index_1] = ''

                else:
                    interpolate_x_indices = []
                    for index_2, value_2 in np.ndenumerate(x_values_overall):
                        index_2 = index_2[0]
                        if x_values_overall[index_2] == x_indv:
                            if index_2 < (len(x_values_overall)-1):
                                interpolate_x_indices = [index_2, index_2+1]
                            else:
                                interpolate_x_indices = [index_2-1, index_2]
                            break
                        elif (    x_values_overall[index_2] < x_indv
                            and x_values_overall[index_2+1] > x_indv
                        ):
                            interpolate_x_indices = [index_2, index_2+1]
                            break

                    x_1 = x_values_overall[interpolate_x_indices[0]]
                    x_2 = x_values_overall[interpolate_x_indices[1]]
                    y_1 = y_values_overall[interpolate_x_indices[0]]
                    y_2 = y_values_overall[interpolate_x_indices[1]]

                    y_1_weight = abs(x_2 - x_indv) / abs(x_2 - x_1)
                    y_2_weight = abs(x_indv - x_1) / abs(x_2 - x_1)

                    y_overall = (y_1*y_1_weight) + (y_2*y_2_weight)

                    propensity = y_indv / y_overall
                    propensities[index_1] = propensity

            x_values_indv_copy = [x for x in x_values_indv_copy if x != '']
            propensities = [y for y in propensities if y != '']

            # Updates dictionary of amino acid propensities
            propensities_dict[aa] = np.array([x_values_indv_copy,
                                              propensities])

            plt.clf()
            plt.figure(figsize=(10, 5))
            axes = plt.gca()
            plt.plot(np.array(x_values_indv_copy), np.array(propensities))
            if save_file is True:
                plt.savefig('{}_{}.svg'.format(aa, file_name))
            plt.show()

    return propensities_dict

In [ ]:
# See notebook prior to 21.02.2021 for equivalent 2D code (for now deleted as insufficient data to run 2D function)
def plot_1d_pairwise_aa_propensities(df, interaction, cont_prop, aa_dict, save_file=False, file_name=''):
    """
    Calculates propensities of amino acid pairs to interact with one another at different values of a continuous
    property of interest
    Input: dataframe of barrel / sandwich properties, name of interaction type of interest (specified via the
           name of the corresponding column in the input dataframe), name of continuous property of interest
           (specified via the name of the corresponding column in the input dataframe), and dictionary of amino
           acid abbreviations 
    """
    df = df[~df[cont_prop].isin(['', 'NaN', 'nan', np.nan])]
    df = df.reset_index(drop=True)

    propensities_dict = OrderedDict()

    neighbouring_pairs_list, prop_dicts = gen_neighbouring_pairs_list(df, interaction, aa_dict, cont_prop)
    prop_dict = prop_dicts[cont_prop]
    propensity_df, frequency_df, normed_frequencies_df = calc_aa_pair_propensities(neighbouring_pairs_list, aa_dict)
    propensity_df = propensity_df.set_index('FASTA', drop=True)

    for aa_1 in list(aa_dict.keys()):
        for aa_2 in list(aa_dict.keys()):
            all_aa_list = []
            aa_1_list = []
            aa_2_list = []
            aa_1_aa_2_list = []

            for aa_pair in list(prop_dict.keys()):
                aas = aa_pair.split('_')[2:]

                all_aa_list.append(prop_dict[aa_pair][0])
                if aa_1 == aas[0]:
                    aa_1_list.append(prop_dict[aa_pair][0])
                if aa_2 == aas[-1]:
                    aa_2_list.append(prop_dict[aa_pair][0])  # Want z_coords of aas interacting with aa_2
                if aa_1 == aas[0] and aa_2 == aas[-1]:
                    aa_1_aa_2_list.append(prop_dict[aa_pair][0])

            if (
                    (not np.isnan(propensity_df[aa_1][aa_2]))
                and (min([len(all_aa_list), len(aa_1_list), len(aa_2_list), len(aa_1_aa_2_list)]) >= 10)    # Threshold is a self-determined "sensible" value - is there a way of determining this threshold empirically?
            ):
                print('Propensity for {} to interact with {}'.format(aa_1, aa_2))

                plt.clf()
                all_aa_array = np.asarray(all_aa_list)
                all_aa_array = all_aa_array.astype(np.float64)
                x_all_aa_list, y_all_aa_list = sns.distributions._statsmodels_univariate_kde(
                    data=all_aa_array, kernel='gau', bw='scott', gridsize=100, cut=3, clip=(-np.inf, np.inf),
                    cumulative=False
                )

                plt.clf()
                aa_1_array = np.asarray(aa_1_list)
                aa_1_array = aa_1_array.astype(np.float64)
                x_aa_1_list, y_aa_1_list = sns.distributions._statsmodels_univariate_kde(
                    data=aa_1_array, kernel='gau', bw='scott', gridsize=100, cut=3, clip=(-np.inf, np.inf),
                    cumulative=False
                )

                plt.clf()
                aa_2_array = np.asarray(aa_2_list)
                aa_2_array = aa_2_array.astype(np.float64)
                x_aa_2_list, y_aa_2_list = sns.distributions._statsmodels_univariate_kde(
                    data=aa_2_array, kernel='gau', bw='scott', gridsize=100, cut=3, clip=(-np.inf, np.inf),
                    cumulative=False
                )

                plt.clf()
                aa_1_aa_2_array = np.asarray(aa_1_aa_2_list)
                aa_1_aa_2_array = aa_1_aa_2_array.astype(np.float64)
                x_aa_1_aa_2_list, y_aa_1_aa_2_list = sns.distributions._statsmodels_univariate_kde(
                    data=aa_1_aa_2_array, kernel='gau', bw='scott', gridsize=100, cut=3, clip=(-np.inf, np.inf),
                    cumulative=False
                )

                x_coord_lists = [x_all_aa_list, x_aa_1_list, x_aa_2_list]
                y_coord_lists = [y_all_aa_list, y_aa_1_list, y_aa_2_list]

                x_aa_1_aa_2_list_copy = list(copy.deepcopy(x_aa_1_aa_2_list))
                interpolated_y_coord_lists = [(['']*len(x_aa_1_aa_2_list_copy)) for num in range(3)]
                propensities = ['']*len(x_aa_1_aa_2_list_copy)

                x_min = max([min(all_aa_list), min(aa_1_list), min(aa_2_list), min(aa_1_aa_2_list)])
                x_max = min([max(all_aa_list), max(aa_1_list), max(aa_2_list), max(aa_1_aa_2_list)])

                for index_1, value_1 in np.ndenumerate(x_aa_1_aa_2_list):
                    index_1 = index_1[0]
                    x_aa_1_aa_2 = x_aa_1_aa_2_list[index_1]
                    y_aa_1_aa_2 = y_aa_1_aa_2_list[index_1]

                    if (
                           x_aa_1_aa_2 < x_min
                        or x_aa_1_aa_2 > x_max
                    ):
                        x_aa_1_aa_2_list_copy[index_1] = ''
                    else:
                        for index_2, value_2 in np.ndenumerate(x_coord_lists):
                            index_2 = index_2[0]
                            x_coord_list = x_coord_lists[index_2]
                            y_coord_list = y_coord_lists[index_2]

                            interpolate_x_indices = []
                            for index_3, value_3 in np.ndenumerate(x_coord_list):
                                index_3 = index_3[0]
                                x_coord = copy.deepcopy(value_3)
                                if x_aa_1_aa_2 == x_coord:
                                    if index_3 != (len(x_coord_list)-1):
                                        interpolate_x_indices = [index_3, index_3+1]
                                    else:
                                        interpolate_x_indices = [index_3-1, index_3]
                                    break
                                elif x_coord_list[index_3] < x_aa_1_aa_2 and x_coord_list[index_3+1] > x_aa_1_aa_2:
                                    interpolate_x_indices = [index_3, index_3+1]
                                    break

                            # Linear interpolation
                            x_1 = x_coord_list[interpolate_x_indices[0]]
                            x_2 = x_coord_list[interpolate_x_indices[1]]
                            y_1 = y_coord_list[interpolate_x_indices[0]]
                            y_2 = y_coord_list[interpolate_x_indices[1]]

                            y_1_weight = abs(x_2 - x_aa_1_aa_2) / abs(x_2 - x_1)
                            y_2_weight = abs(x_aa_1_aa_2 - x_1) / abs(x_2 - x_1)

                            y = (y_1*y_1_weight) + (y_2*y_2_weight)
                            interpolated_y_coord_lists[index_2][index_1] = y


                for index_1, value_1 in enumerate(x_aa_1_aa_2_list_copy):
                    if value_1 != '':
                        y_all_aa = interpolated_y_coord_lists[0][index_1]
                        y_aa_1 = interpolated_y_coord_lists[1][index_1]
                        y_aa_2 = interpolated_y_coord_lists[2][index_1]
                        y_aa_1_aa_2 = y_aa_1_aa_2_list[index_1]
                        y_ratio = ((y_aa_1_aa_2 / y_aa_1)
                                   / (y_aa_2 / y_all_aa))

                        propensity = y_ratio * propensity_df[aa_1][aa_2]
                        propensities[index_1] = propensity

                # Plots kernel density estimates used in the calculation of the pairwise amino acid propensity plot
                plt.clf()
                plt.figure(figsize=(10, 5))
                ax = plt.gca()
                sns.distplot(all_aa_list, hist=False, rug=True, kde_kws={'bw':'scott'}, rug_kws={'alpha': 0.1})
                sns.distplot(aa_1_list, hist=False, rug=True, kde_kws={'bw':'scott'}, rug_kws={'alpha': 0.4})
                sns.distplot(aa_2_list, hist=False, rug=True, kde_kws={'bw':'scott'}, rug_kws={'alpha': 0.4})
                sns.distplot(aa_1_aa_2_list, hist=False, rug=True, kde_kws={'bw':'scott'}, rug_kws={'alpha': 0.8})
                if save_file is True:
                    plt.savefig('{}{}_{}_kdes.svg'.format(aa_1, aa_2, file_name))
                plt.show()

                # Plots pairwise amino acid propensities vs. z-coordinate
                x_aa_1_aa_2_list_copy = [x for x in x_aa_1_aa_2_list_copy if x != '']
                propensities = [y for y in propensities if y != '']

                plt.clf()
                plt.figure(figsize=(10, 5))
                ax = plt.gca()
                plt.plot(np.array(x_aa_1_aa_2_list_copy), np.array(propensities))
                if save_file is True:
                    plt.savefig('{}{}_{}_propensities.svg'.format(aa_1, aa_2, file_name))
                plt.show()

                # Updates dictionary of amino acid propensities
                propensities_dict['{}_{}'.format(aa_1, aa_2)] = np.array([x_aa_1_aa_2_list_copy,
                                                                          propensities])
                
    return propensities_dict

In [ ]:
def plot_discrete_prop_frequency_kdes(df, cont_prop, discrete_props, save_file=False, file_name=''):
    """
    Plots kernel density estimate of the distribution of a discrete property (e.g. the number of interactions of a
    particular type (e.g. hydrogen bonds) formed by an amino acid)  vs. a continuous property (typically z-coordinate)
    Input: dataframe of barrel / sandwich properties, name of continuous property of interest (specified via the
           name of the corresponding column in the input dataframe), list of discrete properties (each discrete
           property is considered individually, in a separate kde plot), whether to save a copy of each plot and if
           so the base file name to save them under
    """
    df = df[~df[cont_prop].isin(['', 'NaN', 'nan', np.nan])]
    df = df.reset_index(drop=True)

    domain_res_ids_list = []
    for row in range(df.shape[0]):
        domain_res_id = df['domain_ids'][row] + df['res_ids'][row]
        domain_res_ids_list.append(domain_res_id)

    for discrete_prop in discrete_props:
        print(discrete_prop)

        cont_prop_distribution = []
        for row in range(df.shape[0]):
            interacting_domain_neighbours = []

            for res_id in df[discrete_prop][row]:
                domain_res_id = df['domain_ids'][row] + res_id
                # Only considers interactions with residues that are also within the domain
                if domain_res_id in domain_res_ids_list:
                    interacting_domain_neighbours.append(domain_res_id)

            interaction_num = len(interacting_domain_neighbours)
            count = 0
            while count < interaction_num:
                cont_prop_distribution.append(df[cont_prop][row])
                count += 1

        plt.clf()
        plt.figure(figsize=(10, 5))
        sns.distplot(pd.DataFrame({cont_prop: cont_prop_distribution}), hist=False, rug=True,
                     kde_kws={'bw':'scott'}, rug_kws={'alpha': 0.1})
        if save_file is True:
            plt.savefig('{}_{}.svg'.format(discrete_prop, file_name))
        plt.show()

In [ ]:
def plot_prop_vs_prop(df, cont_prop_1, cont_prop_dict_2, cont_prop_2, bw_1, bw_2):
    """
    Plots the 2D kernel desity estimate of two continuous properties
    Input: dataframe of barrel / sandwich properties, name of first continuous property of interest (specified via
           the name of the corresponding column in the input dataframe), dictionary of amino acid identities vs.
           their values of the second continuous property of interest, the name of the second continuous property
           of interest, the kernel bandwidth of the first property and the kernel bandwidth of the second property
    """
    cont_prop_2_list = ['']*df.shape[0]
    unprocessed_indices = []

    for row in range(df.shape[0]):
        fasta = df['fasta_seq'][row]

        if fasta in cont_prop_dict_2:
            cont_prop_2_list[row] = cont_prop_dict_2[fasta]
        else:
            unprocessed_indices.append(row)

    cont_prop_1_list = [val for index, val in enumerate(df[cont_prop_1].tolist()) if not index in unprocessed_indices]
    cont_prop_2_list = [val for index, val in enumerate(cont_prop_2_list) if not index in unprocessed_indices]

    prop_df = pd.DataFrame({cont_prop_1: cont_prop_1_list,
                            cont_prop_2: cont_prop_2_list})

    plt.clf()
    plt.figure(figsize=(10, 10))
    sns.kdeplot(data=prop_df[cont_prop_1], data2=prop_df[cont_prop_2], bw=(bw_1,bw_2), shade=True, cbar=True)
    # plt.scatter(prop_df[cont_prop_1], prop_df[cont_prop_2], marker='.', c='y')
    plt.show()

In [ ]:
def plot_prop_vs_dihedral(df, cont_prop, dihedral):
    """
    Plots the 2D kernel desity estimate of a continuous property of interest vs. a dihedral angle
    Input: dataframe of barrel / sandwich properties, name of continuous property of interest (specified via
           the name of the corresponding column in the input dataframe), name of dihedral angle of interest
           (specified via the name of the corresponding column in the input dataframe)
    """
    df = df[~df[dihedral].isin(['', 'NaN', 'nan', np.nan])]
    df = df.reset_index(drop=True)

    dihedral_df = pd.DataFrame({cont_prop: df[cont_prop].tolist(),
                                dihedral: df[dihedral].tolist()})

    plt.clf()
    plt.figure(figsize=(10, 10))
    sns.kdeplot(data=dihedral_df[cont_prop], data2=dihedral_df[dihedral], bw='scott', shade=True, cbar=True)
    plt.show()

In [ ]:
def calc_dist(df, cont_prop):
    """
    Generates ordered array of a (numeric) property of interest in the input dataframe, without '', 'NaN', 'nan'
    and np.nan values
    Input: dataframe of barrel / sandwich properties, name of property of interest (specified via the name of
           the corresponding column in the input dataframe)
    Returns: ordered numpy array of the property of interest
    """
    cont_prop_vals = df[cont_prop].tolist()
    cont_prop_vals = remove_nan(cont_prop_vals)
    cont_prop_vals = np.array(sorted(cont_prop_vals))

    return cont_prop_vals


def gen_random_array(ordered_array):
    """
    Generates a random array by sampling, with replacement, an input array
    Input: ordered array
    Returns: ordered array (of the same length as the input array) generated by sampling the input array with
             replacement
    """
    random_array = np.empty(len(ordered_array),)

    for num in range(len(ordered_array)):
        rand_index = random.randint(0, (len(ordered_array)-1))
        rand_z_coord = ordered_array[rand_index]
        random_array[num] = rand_z_coord

    return np.sort(random_array)


def run_ks_2samp_test(df, cont_prop, aa, bootstrap):
    """
    Runs the 1D 2 sample Kolmogorov-Smirnov test with scipy to calculate the likelihood that two samples come from
    the same distribution (where sample 1 is the distribution of an individual amino acid with respect to a
    continuous property of interest (typically z-coordinate), and sample 2 is the distribution of all amino acids
    with respect to that property)
    Input: dataframe of barrel / sandwich properties, name of the continuous property of interest (specified via
           the name of the corresponding column in the input dataframe), the name of the individual amino acid, and
           whether this test is being run as part of a bootstrap test or not
    """
    df = df[~df[cont_prop].isin(['', 'NaN', 'nan', np.nan])]
    df = df.reset_index(drop=True)

    aa_df = df[df['fasta_seq'] == aa]
    aa_df = aa_df.reset_index(drop=True)

    indv_aa_cont_prop_vals = remove_nan(aa_df[cont_prop].tolist())
    indv_aa_cont_prop_vals = np.array(sorted(indv_aa_cont_prop_vals))

    # If performing bootstrap analysis, generate a random array from the df[cont_prop] distribution
    if bootstrap:
        indv_aa_cont_prop_vals = gen_random_array(indv_aa_cont_prop_vals)

    D, p = scipy.stats.ks_2samp(indv_aa_cont_prop_vals, calc_dist(df, cont_prop))

    return D, p


def initial_ks_test(df, cont_prop, ks_func, aa_dict):
    """
    Wrapper to run the 1D 2 sample Kolmogorov-Smirnov test for the original (i.e. not resampled) individual amino
    acid distribution and the overall amino acid distribution
    Input: dataframe of barrel / sandwich properties, name of the continuous property of interest (specified via
           the name of the corresponding column in the input dataframe), the function to run the KS test, and
           dictionary of amino acid abbreviations
    Returns: dictionaries of D and p values output from KS test
    """
    D_dict = OrderedDict()
    p_dict = OrderedDict()

    for aa in list(aa_dict.keys()):
        D, p = ks_func(df, cont_prop, aa, False)
        D_dict[aa] = [D]
        p_dict[aa] = [p]

    return D_dict, p_dict

def bootstrap_ks_test(df, cont_prop, bootstrap_num, ks_func, aa_dict):
    """
    Wrapper to run the 1D 2 sample Kolmogorov-Smirnov test a specified number of times on different resamplings
    of the original individual amino acid distribution and the overall amino acid distribution
    Input: dataframe of barrel / sandwich properties, name of the continuous property of interest (specified via
           the name of the corresponding column in the input dataframe), the number of KS tests to run (i.e. the
           size of the bootstrap), the function to run the KS test, dictionary of amino acid abbreviations
    Returns: dictionaries of D and p values output from KS test
    """
    D_dict = OrderedDict()
    p_dict = OrderedDict()

    for aa in list(aa_dict.keys()):
        D_dict[aa] = []
        p_dict[aa] = []

    for num in range(bootstrap_num):
        for aa in list(aa_dict.keys()):
            D, p = ks_func(df, cont_prop, aa, True)
            D_dict[aa].append(D)
            p_dict[aa].append(p)

    return D_dict, p_dict


def calc_95_conf_limits(initial_stat_dict, bootstrap_stat_dict):
    """
    Calculates 95% confidence limits on D / p from a bootstrap sample
    Input: dictionary of D / p values for original individual amino acid samples, dictionary of D / p values for
           bootstrapped individual amino acid samples
    Returns: dictionary of 95% confidence intervals for D / p
    """
    conf_int_dict = OrderedDict()

    for aa in list(initial_stat_dict.keys()):
        initial_stat = initial_stat_dict[aa][0]
        bootstrap_stat_values = np.array(bootstrap_stat_dict[aa])

        lower_percentile = np.percentile(bootstrap_stat_values, 2.5)
        upper_percentile = np.percentile(bootstrap_stat_values, 97.5)

        conf_int_dict[aa] = [lower_percentile, initial_stat, upper_percentile]
        
    return conf_int_dict


def draw_swarm_plot(aa, stat_dict, y_label, hue_label='', save_file=False, file_name=''):
    """
    Generates swarm plot of D / p value with bootstrapped 95% confidence intervals vs. amino acid identity
    Input: amino acid name (set to 'all' if stat_dict is a nested dictionary for all aas in the dataset),
           dictionary of D or p values, label for y-axis (e.g. 'D' or 'p'), label for property to split by hue,
           whether to save a copy of the plot and if so the file name to save it under
    """

    x_values = []
    y_values = []
    if aa.lower() == 'all':
        hue_values = []

    if aa.lower() == 'all':
        for x in list(stat_dict.keys()):
            for hue in list(stat_dict[x].keys()):
                for y in stat_dict[x][hue]:
                    x_values.append(x)
                    y_values.append(y)
                    hue_values.append(hue)
    else:
        for x in list(stat_dict.keys()):
            for y in stat_dict[x]:
                x_values.append(x)
                y_values.append(y)

    # Generates "melted" df
    if aa.lower() == 'all':
        df = pd.DataFrame({'Num_bootstrap_samples': x_values,
                           '{}'.format(y_label): y_values,
                           '{}'.format(hue_label): hue_values})
    else:
        df = pd.DataFrame({'Num_bootstrap_samples': x_values,
                           '{}'.format(y_label): y_values})

    plt.clf()
    plt.figure(figsize=(8, 5))
    if aa.lower() == 'all':
        sns.swarmplot(x='Num_bootstrap_samples', y=y_label, hue=hue_label, dodge=True, data=df)
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    else:
        sns.swarmplot(x='Num_bootstrap_samples', y=y_label, data=df)
    if save_file is True:
        plt.savefig('{}_{}.svg'.format(aa, file_name))
    plt.show()


def draw_strip_plot(stat_dict, y_label, save_file=False, file_name=''):
    """
    Generates strip plot of D / p value with 95% confidence intervals vs. amino acid identity
    Input: dictionary of D or p values, label for y-axis (e.g. 'D' or 'p'), whether to save a copy of the generated
           plot and if so the file name to save it under
    """

    orig_x_values = []
    orig_y_values = []
    percentile_x_values = []
    percentile_y_values = []

    for aa in list(stat_dict.keys()):
        for i, y in enumerate(stat_dict[aa]):
            if i == 1:
                orig_x_values.append(aa)
                orig_y_values.append(y)
            else:
                percentile_x_values.append(aa)
                percentile_y_values.append(y)

    # "Melted" df of aa vs. stat (either p or D)
    orig_df = pd.DataFrame({'Amino acid': orig_x_values,
                            '{}'.format(y_label): orig_y_values})
    percentile_df = pd.DataFrame({'Amino acid': percentile_x_values,
                              '{}'.format(y_label): percentile_y_values})

    plt.clf()
    plt.figure(figsize=(12, 9))
    sns.barplot(x='Amino acid', y=y_label, data=orig_df)
    sns.stripplot(x='Amino acid', y=y_label, data=percentile_df, jitter=False, edgecolor='k', linewidth=2)
    if save_file is True:
        plt.savefig('{}.svg'.format(file_name))
    plt.show()


def iterate_bootstrap_conf_limits(df, cont_prop, ks_func, bootstrap_range, aa_dict):
    """
    Performs KS test with increasing number of bootstrap samples and plots output to allow user to determine when the
    confidence intervals converge (and so the minimum number of bootstrap samples they need to run)
    Input: dataframe of barrel / sandwich properties, name of the continuous property of interest (typically
           z-coordinate) (specified via the name of the corresponding column in the input dataframe), the function
           to run the KS test, list of numbers of bootstrap samples to test (e.g. [100, 300, 1000, 3000, 10000, 30000,
           100000]), dictionary of amino acid abbreviations
    """
    initial_D_dict, initial_p_dict = initial_ks_test(df, cont_prop, ks_func, aa_dict)

    bootstrap_D_dict = OrderedDict()
    bootstrap_p_dict = OrderedDict()

    for num in bootstrap_range:
        print(num)
        D_dict, p_dict = bootstrap_ks_test(df, cont_prop, num, ks_func, aa_dict)
        bootstrap_D_dict[num] = D_dict
        bootstrap_p_dict[num] = p_dict

    bootstrap_conf_intv_D_dict = OrderedDict()
    bootstrap_conf_intv_p_dict = OrderedDict()
    for num in bootstrap_range:
        conf_int_D_dict = calc_95_conf_limits(initial_D_dict, bootstrap_D_dict[num])
        bootstrap_conf_intv_D_dict[num] = conf_int_D_dict
        conf_int_p_dict = calc_95_conf_limits(initial_p_dict, bootstrap_p_dict[num])
        bootstrap_conf_intv_p_dict[num] = conf_int_p_dict

    draw_swarm_plot('All', bootstrap_conf_intv_p_dict, 'p', 'Amino acid')
    for aa in list(aa_dict.keys()):
        aa_conf_int_p_dict = OrderedDict()
        for num in bootstrap_range:
            aa_conf_int_p_dict[num] = bootstrap_conf_intv_p_dict[num][aa]
        
        print(aa)
        draw_swarm_plot(aa, aa_conf_int_p_dict, 'p')

In [ ]:
def calc_pair_proportions(df, pair_type, aa_dict):
    """
    For every pairwise combination of amino acids listed in the input dictionary of amino acid abbreviations, aa1aa2,
    calculates proportions of amino acid pairs at a location of interest (e.g. HB, or NHB, pairs) that are of that
    identity. Note no pair inversion.
    Input: dataframe of sandwich / barrel properties, name of amino acid pair type of interest (specified via the
           name of its corresponding column in the input dataframe), and dictionary of amino acid abbreviations
    Returns: dictionary of aa1aa2 pair count, total number of amino acid pairs of type of interest
    """
    neighbouring_pairs_list = gen_neighbouring_pairs_list(df, pair_type, aa_dict)

    pairs_count_dict = OrderedDict()
    for aa_pair in itertools.combinations_with_replacement(list(aa_dict.keys()), 2):
        aa_pair = '{}_{}'.format(aa_pair[0], aa_pair[1])
        pairs_count_dict[aa_pair] = 0

    for aa_pair in neighbouring_pairs_list:
        aa_1 = aa_pair.split('_')[0]
        aa_2 = aa_pair.split('_')[1]
        aa_pair_fwd = copy.deepcopy(aa_pair)
        aa_pair_rev = aa_pair_fwd[::-1]

        if (    aa_1 != aa_2
            and aa_pair_fwd in list(pairs_count_dict.keys())
            and aa_pair_rev in list(pairs_count_dict.keys())
        ):
            sys.exit('Error with standard error of proporton calculation')

        if aa_pair_fwd in list(pairs_count_dict.keys()):
            pairs_count_dict[aa_pair_fwd] += 1
        elif aa_pair_rev in list(pairs_count_dict.keys()):
            pairs_count_dict[aa_pair_rev] += 1

    pairs_total = len(neighbouring_pairs_list)
    for aa_pair in list(pairs_count_dict.keys()):
        pairs_count_dict[aa_pair] = pairs_count_dict[aa_pair] / pairs_total

    return pairs_count_dict, pairs_total


def gen_dek_sandwich_dicts(aa_dict, hb_or_nhb):
    """
    Reads beta-sandwich HB / NHB count data from Dek's 1998 paper (which is stored in two separate csv files in
    the BetaStats directory) into a dictionary
    Input: dictionary of amino acid abbreviations, and whether to read in HB (hb_or_nhb = 'hb') or NHB
           (hb_or_nhb = 'nhb') count data
    Returns: dictionary of amino acid pair proportions, total number of HB (hb_or_nhb = 'hb') or NHB
             (hb_or_nhb = 'nhb') amino acid pairs
    """
    pairs_count_dict = OrderedDict()
    for aa_pair in itertools.combinations_with_replacement(list(aa_dict.keys()), 2):
        aa_pair = '{}_{}'.format(aa_pair[0], aa_pair[1])
        pairs_count_dict[aa_pair] = 0
        
    file_name = '../../Beta_{}_count_data_Hutchinson_et_al_1998.csv'.format(hb_or_nhb.upper())
    df = pd.read_csv(file_name, header=0, index_col=0)
    for col in df.columns.values.tolist():
        df[col] = pd.to_numeric(df[col].tolist())

    pairs_total = 0
    for aa_pair in list(pairs_count_dict.keys()):
        aa_1 = aa_pair.split('_')[0]
        aa_2 = aa_pair.split('_')[1]

        count = df[aa_1][aa_2]
        pairs_total += count
        pairs_count_dict[aa_pair] = count

    for aa_pair in list(pairs_count_dict.keys()):
        pairs_count_dict[aa_pair] = pairs_count_dict[aa_pair] / pairs_total

    return pairs_count_dict, pairs_total


def calc_std_error_proportion(aa_dict, hb_pairs_count_dict, nhb_pairs_count_dict, hb_pairs_total,
                              nhb_pairs_total):
    """
    Calculates standard error of proportion of amino acid pairs that form HB as compared with NHB pairs. These
    results can be used to determine if there is a significant difference between the proportion of aa1aa2 pairs
    at HB positions as compared to NHB positions.
    Input: dictionary of amino acid abbreviations, dictionary of hydrogen bonding pair probabilities, dictionary
           of non-hydrogen-bonding pair probabilities, total number of hydrogen bonding pairs, total number of
           non-hydrogen-bonding pairs
    dataframe of barrel / sandwich properties, dictionary of amino acid abbreviations
    Returns: dictionary of z scores, dictionary of ratios of proportion of aa1aa2 pairs at HB sites to proportion
             of aa1aa2 pairs at NHB sites
    """
    z_score_dict = OrderedDict()
    ratios_dict = OrderedDict()
    for aa_pair in itertools.combinations_with_replacement(list(aa_dict.keys()), 2):
        aa_pair = '{}_{}'.format(aa_pair[0], aa_pair[1])
        z_score_dict[aa_pair] = np.nan
        ratios_dict[aa_pair] = np.nan

    for aa_pair in list(hb_pairs_count_dict):
        p_hb = hb_pairs_count_dict[aa_pair]
        p_nhb = nhb_pairs_count_dict[aa_pair]
        
        if p_hb > 0 and p_nhb > 0:
            variance_hb = (p_hb*(1-p_hb)) / hb_pairs_total
            variance_nhb = (p_nhb*(1-p_nhb)) / nhb_pairs_total

            std_dev = (variance_hb + variance_nhb)**0.5
            z = (p_hb - p_nhb) / std_dev
            z_score_dict[aa_pair] = z
            
            ratios_dict[aa_pair] = p_hb / p_nhb

    return z_score_dict, ratios_dict


def print_significant_z_scores(z_score_dict):
    """
    Prints z-scores above / below 95 and 99% confidence limits
    Input: dictionary of z scores
    """
    for aa_pair, score in z_score_dict.items():
        if score > 2.58:
            print('Outside 99% confidence limit, favours HB: {}'.format(aa_pair))
        elif 1.96 < score <= 2.58:
            print('Outside 95% confidence limit, favours HB: {}'.format(aa_pair))
        elif score < -2.58:
            print('Outside 99% confidence limit, favours NHB: {}'.format(aa_pair))
        elif -2.58 <= score < -1.96:
            print('Outside 95% confidence limit, favours NHB: {}'.format(aa_pair))


def calc_z_diff(z_score_dict, dek_sandwich_z_dict):
    """
    Calculates the difference in area underneath the normal distribution curve between two z-scores
    Input: first dictionary of z scores, second dictionary of z scores
    Returns: dictionary of differences
    """
    z_diff_dict = OrderedDict()

    for aa_pair in list(z_score_dict.keys()):
        z_score_old = dek_sandwich_z_dict[aa_pair]
        z_score_new = z_score_dict[aa_pair]
        
        p_old = scipy.stats.norm.cdf(z_score_old)
        p_new = scipy.stats.norm.cdf(z_score_new)
        
        area_diff = abs(p_new - p_old)
        z_diff_dict[aa_pair] = area_diff

    return z_diff_dict

## Load dataframe

In [ ]:
sandwich_file_loc = (
    '/Users/ks17361/Lab_work_Dek_Woolfson/Beta_structure/Parametrisation/BetaStats/'
    'Beta_res_CATH_2.60.40.10_resn_1.6_rfac_0.2_ba_seqid_40.pkl'
)

In [ ]:
sandwich_df = pd.read_pickle(sandwich_file_loc)
sandwich_df = sandwich_df[sandwich_df['fasta_seq'].isin(list(aa_dict.keys()))]
sandwich_df = sandwich_df.reset_index(drop=True)

In [ ]:
sandwich_df.head()

In [ ]:
if not os.path.isdir('Sandwich_plots'):
    os.mkdir('Sandwich_plots')
os.chdir('Sandwich_plots/')

### Amino acid distribution in beta-sandwiches

In [ ]:
sandwich_distribution = calc_distribution(sandwich_df, aa_dict)

plt.clf()
plt.figure(figsize=(20, 8))
sns.barplot(x='FASTA', y='Normalised frequency', data=sandwich_distribution)
plt.savefig('Amino_acid_distribution.svg')
plt.show()

In [ ]:
for aa in list(aa_dict.keys()):
    print(aa + ': ' + str(sandwich_df['fasta_seq'].tolist().count(aa)))

How does this distribution compare to those previously determined?

In [ ]:
levitt_dict = OrderedDict({'A': 0.900, 'R': 0.990, 'N': 0.760, 'D': 0.720, 'C': 0.740, 'Q': 0.800, 'E': 0.750,
                           'G': 0.920, 'H': 1.080, 'I': 1.450, 'L': 1.020, 'K': 0.770, 'M': 0.970, 'F': 1.320,
                           'P': 0.640, 'S': 0.950, 'T': 1.210, 'W': 1.140, 'Y': 1.250, 'V': 1.490})
levitt_df = pd.DataFrame({'FASTA': list(levitt_dict.keys()),
                          'Propensity': list(levitt_dict.values())})

plt.clf()
plt.figure(figsize=(20, 8))
sns.barplot(x='FASTA', y='Propensity', data=levitt_df)
plt.savefig('Levitt_beta_distribution.svg')
plt.show()

In [ ]:
chou_fasman_dict = OrderedDict({'A': 0.830, 'R': 0.930, 'N': 0.890, 'D': 0.540, 'C': 1.190, 'Q': 1.100, 'E': 0.370,
                                'G': 0.750, 'H': 0.870, 'I': 1.600, 'L': 1.300, 'K': 0.740, 'M': 1.050, 'F': 1.380, 
                                'P': 0.550, 'S': 0.750, 'T': 1.190, 'W': 1.370, 'Y': 1.470, 'V': 1.700})
chou_fasman_df = pd.DataFrame({'FASTA': list(chou_fasman_dict.keys()),
                               'Propensity': list(chou_fasman_dict.values())})

plt.clf()
plt.figure(figsize=(20, 8))
sns.barplot(x='FASTA', y='Propensity', data=chou_fasman_df)
plt.savefig('Chou_Fasman_beta_distribution.svg')
plt.show()

## Discrete propensity calculations

In [ ]:
if not os.path.isdir('Discrete_propensities_indv_aas'):
    os.mkdir('Discrete_propensities_indv_aas')
os.chdir('Discrete_propensities_indv_aas')

### Interior / exterior propensity

In [ ]:
int_ext_propensity, int_ext_frequency, int_ext_normed_frequencies = calc_indv_property_propensities(
    sandwich_df, 'int_ext', aa_dict
)

In [ ]:
int_ext_propensity

In [ ]:
int_ext_frequency

In [ ]:
plot_bar_graphs(int_ext_propensity, 'Propensity', True, 'Int_ext_discrete_propensity')

In [ ]:
plot_bar_graphs(int_ext_frequency, 'Frequency', True, 'Int_ext_discrete_frequency')

In [ ]:
plot_heat_map(int_ext_propensity)

In [ ]:
plot_heat_map(int_ext_frequency)

In [ ]:
(
    int_ext_bootstrap_propensity_dict, int_ext_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    sandwich_df, 'int_ext', aa_dict, [10, 30, 100, 300, 1000], calc_indv_property_propensities, int_ext_propensity,
    int_ext_frequency
)

In [ ]:
(
    int_ext_conf_intv_propensity_dict, int_ext_conf_intv_frequency_dict, int_ext_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    sandwich_df, 'int_ext', aa_dict, 300, calc_indv_property_propensities
)

In [ ]:
lower_percentile_int_ext_propensity_dict, upper_percentile_int_ext_propensity_dict = gen_95_conf_intervals(
    int_ext_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    int_ext_propensity, lower_percentile_int_ext_propensity_dict, upper_percentile_int_ext_propensity_dict,
    'Propensity', True, 'Int_ext_discrete_propensity_95_conf'
)

In [ ]:
lower_percentile_int_ext_frequency_dict, upper_percentile_int_ext_frequency_dict = gen_95_conf_intervals(
    int_ext_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    int_ext_frequency, lower_percentile_int_ext_frequency_dict, upper_percentile_int_ext_frequency_dict, 'Frequency',
    True, 'Int_ext_discrete_frequency_95_conf'
)

### Edge / central propensity

In [ ]:
edg_cen_propensity, edg_cen_frequency, edg_cen_normed_frequencies = calc_indv_property_propensities(
    sandwich_df, 'edge_or_central', aa_dict
)

In [ ]:
edg_cen_propensity

In [ ]:
edg_cen_frequency

In [ ]:
plot_bar_graphs(edg_cen_propensity, 'Propensity', True, 'Edg_cen_discrete_propensity')

In [ ]:
plot_bar_graphs(edg_cen_frequency, 'Frequency', True, 'Edg_cen_discrete_frequency')

In [ ]:
plot_heat_map(edg_cen_propensity)

In [ ]:
plot_heat_map(edg_cen_frequency)

In [ ]:
(
    edg_cen_bootstrap_propensity_dict, edg_cen_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    sandwich_df, 'edge_or_central', aa_dict, [10, 30, 100, 300, 1000], calc_indv_property_propensities,
    edg_cen_propensity, edg_cen_frequency
)

In [ ]:
(
    edg_cen_conf_intv_propensity_dict, edg_cen_conf_intv_frequency_dict, edg_cen_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    sandwich_df, 'edge_or_central', aa_dict, 300, calc_indv_property_propensities
)

In [ ]:
lower_percentile_edg_cen_propensity_dict, upper_percentile_edg_cen_propensity_dict = gen_95_conf_intervals(
    edg_cen_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    edg_cen_propensity, lower_percentile_edg_cen_propensity_dict, upper_percentile_edg_cen_propensity_dict,
    'Propensity', True, 'Edg_cen_discrete_propensity_95_conf'
)

In [ ]:
lower_percentile_edg_cen_frequency_dict, upper_percentile_edg_cen_frequency_dict = gen_95_conf_intervals(
    edg_cen_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    edg_cen_frequency, lower_percentile_edg_cen_frequency_dict, upper_percentile_edg_cen_frequency_dict, 'Frequency',
    True, 'Edg_cen_discrete_frequency_95_conf'
)

### Core / surface propensity

In [ ]:
core_surf_propensity, core_surf_frequency, core_surf_normed_frequencies = calc_indv_property_propensities(
    sandwich_df, 'core_surf', aa_dict
)

In [ ]:
core_surf_propensity

In [ ]:
core_surf_frequency

In [ ]:
plot_bar_graphs(core_surf_propensity, 'Propensity', True, 'Core_surf_discrete_propensity')

In [ ]:
plot_bar_graphs(core_surf_frequency, 'Frequency', True, 'Core_surf_discrete_frequency')

In [ ]:
plot_heat_map(core_surf_propensity)

In [ ]:
plot_heat_map(core_surf_frequency)

In [ ]:
(
    core_surf_bootstrap_propensity_dict, core_surf_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    sandwich_df, 'core_surf', aa_dict, [10, 30, 100, 300, 1000], calc_indv_property_propensities,
    core_surf_propensity, core_surf_frequency
)

In [ ]:
(
    core_surf_conf_intv_propensity_dict, core_surf_conf_intv_frequency_dict,
    core_surf_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    sandwich_df, 'core_surf', aa_dict, 300, calc_indv_property_propensities
)

In [ ]:
lower_percentile_core_surf_propensity_dict, upper_percentile_core_surf_propensity_dict = gen_95_conf_intervals(
    core_surf_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    core_surf_propensity, lower_percentile_core_surf_propensity_dict, upper_percentile_core_surf_propensity_dict,
    'Propensity', True, 'Core_surf_discrete_propensity_95_conf'
)

In [ ]:
lower_percentile_core_surf_frequency_dict, upper_percentile_core_surf_frequency_dict = gen_95_conf_intervals(
    core_surf_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    core_surf_frequency, lower_percentile_core_surf_frequency_dict, upper_percentile_core_surf_frequency_dict,
    'Frequency', True, 'Core_surf_discrete_frequency_95_conf'
)

### All combinations propensities

### Edge / central and interior / exterior

In [ ]:
(edg_cen_int_ext_propensity, edg_cen_int_ext_frequency, edg_cen_int_ext_normed_frequencies
) = calc_combined_property_propensities(
    sandwich_df, ['edge_or_central', 'int_ext'], aa_dict
)

In [ ]:
edg_cen_int_ext_propensity

In [ ]:
edg_cen_int_ext_frequency

In [ ]:
plot_bar_graphs(edg_cen_int_ext_propensity, 'Propensity', True, 'Edg_cen_int_ext_discrete_propensity')

In [ ]:
plot_bar_graphs(edg_cen_int_ext_frequency, 'Frequency', True, 'Edg_cen_int_ext_discrete_frequency')

In [ ]:
plot_heat_map(edg_cen_int_ext_propensity)

In [ ]:
plot_heat_map(edg_cen_int_ext_frequency)

In [ ]:
(
    edg_cen_int_ext_bootstrap_propensity_dict, edg_cen_int_ext_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    sandwich_df, ['edge_or_central', 'int_ext'], aa_dict, [10, 30, 100, 300, 1000],
    calc_combined_property_propensities, edg_cen_int_ext_propensity, edg_cen_int_ext_frequency
)

In [ ]:
(
    edg_cen_int_ext_conf_intv_propensity_dict, edg_cen_int_ext_conf_intv_frequency_dict,
    edg_cen_int_ext_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    sandwich_df, ['edge_or_central', 'int_ext'], aa_dict, 300, calc_combined_property_propensities
)

In [ ]:
(
    lower_percentile_edg_cen_int_ext_propensity_dict, upper_percentile_edg_cen_int_ext_propensity_dict
) = gen_95_conf_intervals(
    edg_cen_int_ext_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    edg_cen_int_ext_propensity, lower_percentile_edg_cen_int_ext_propensity_dict,
    upper_percentile_edg_cen_int_ext_propensity_dict, 'Propensity', True,
    'Edg_cen_int_ext_discrete_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_edg_cen_int_ext_frequency_dict, upper_percentile_edg_cen_int_ext_frequency_dict
) = gen_95_conf_intervals(
    edg_cen_int_ext_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    edg_cen_int_ext_frequency, lower_percentile_edg_cen_int_ext_frequency_dict,
    upper_percentile_edg_cen_int_ext_frequency_dict, 'Frequency', True, 'Edg_cen_int_ext_discrete_frequency_95_conf'
)

### Core / surface and interior / exterior propensities

In [ ]:
(core_surf_int_ext_propensity, core_surf_int_ext_frequency, core_surf_int_ext_normed_frequencies
) = calc_combined_property_propensities(
    sandwich_df, ['core_surf', 'int_ext'], aa_dict
)

In [ ]:
core_surf_int_ext_propensity

In [ ]:
core_surf_int_ext_frequency

In [ ]:
plot_bar_graphs(core_surf_int_ext_propensity, 'Propensity', True, 'Core_surf_int_ext_discrete_propensity')

In [ ]:
plot_bar_graphs(core_surf_int_ext_frequency, 'Frequency', True, 'Core_surf_int_ext_discrete_frequency')

In [ ]:
plot_heat_map(core_surf_int_ext_propensity)

In [ ]:
plot_heat_map(core_surf_int_ext_frequency)

In [ ]:
(
    core_surf_int_ext_bootstrap_propensity_dict, core_surf_int_ext_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    sandwich_df, ['core_surf', 'int_ext'], aa_dict, [10, 30, 100, 300, 1000],
    calc_combined_property_propensities, core_surf_int_ext_propensity, core_surf_int_ext_frequency
)

In [ ]:
(
    core_surf_int_ext_conf_intv_propensity_dict, core_surf_int_ext_conf_intv_frequency_dict,
    core_surf_int_ext_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    sandwich_df, ['core_surf', 'int_ext'], aa_dict, 300, calc_combined_property_propensities
)

In [ ]:
(
    lower_percentile_core_surf_int_ext_propensity_dict, upper_percentile_core_surf_int_ext_propensity_dict
) = gen_95_conf_intervals(
    core_surf_int_ext_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    core_surf_int_ext_propensity, lower_percentile_core_surf_int_ext_propensity_dict,
    upper_percentile_core_surf_int_ext_propensity_dict, 'Propensity', True,
    'Core_surf_int_ext_discrete_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_core_surf_int_ext_frequency_dict, upper_percentile_core_surf_int_ext_frequency_dict
) = gen_95_conf_intervals(
    core_surf_int_ext_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    core_surf_int_ext_frequency, lower_percentile_core_surf_int_ext_frequency_dict,
    upper_percentile_core_surf_int_ext_frequency_dict, 'Frequency', True,
    'Core_surf_int_ext_discrete_frequency_95_conf'
)

### Core / surface and edge / central propensities

In [ ]:
(core_surf_edg_cen_propensity, core_surf_edg_cen_frequency, core_surf_edg_cen_normed_frequencies
) = calc_combined_property_propensities(
    sandwich_df, ['core_surf', 'edge_or_central'], aa_dict
)

In [ ]:
core_surf_edg_cen_propensity

In [ ]:
core_surf_edg_cen_frequency

In [ ]:
plot_bar_graphs(core_surf_edg_cen_propensity, 'Propensity', True, 'Core_surf_edg_cen_discrete_propensity')

In [ ]:
plot_bar_graphs(core_surf_edg_cen_frequency, 'Frequency', True, 'Core_surf_edg_cen_discrete_frequency')

In [ ]:
plot_heat_map(core_surf_edg_cen_propensity)

In [ ]:
plot_heat_map(core_surf_edg_cen_frequency)

In [ ]:
(
    core_surf_edg_cen_bootstrap_propensity_dict, core_surf_edg_cen_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    sandwich_df, ['core_surf', 'edge_or_central'], aa_dict, [10, 30, 100, 300, 1000],
    calc_combined_property_propensities, core_surf_edg_cen_propensity, core_surf_edg_cen_frequency
)

In [ ]:
(
    core_surf_edg_cen_conf_intv_propensity_dict, core_surf_edg_cen_conf_intv_frequency_dict,
    core_surf_edg_cen_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    sandwich_df, ['core_surf', 'edge_or_central'], aa_dict, 300, calc_combined_property_propensities
)

In [ ]:
(
    lower_percentile_core_surf_edg_cen_propensity_dict, upper_percentile_core_surf_edg_cen_propensity_dict
) = gen_95_conf_intervals(
    core_surf_edg_cen_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    core_surf_edg_cen_propensity, lower_percentile_core_surf_edg_cen_propensity_dict,
    upper_percentile_core_surf_edg_cen_propensity_dict, 'Propensity', True,
    'Core_surf_edg_cen_discrete_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_core_surf_edg_cen_frequency_dict, upper_percentile_core_surf_edg_cen_frequency_dict
) = gen_95_conf_intervals(
    core_surf_edg_cen_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    core_surf_edg_cen_frequency, lower_percentile_core_surf_edg_cen_frequency_dict,
    upper_percentile_core_surf_edg_cen_frequency_dict, 'Frequency', True,
    'Core_surf_edg_cen_discrete_frequency_95_conf'
)

### Defines interior, exterior, edge and central sub-dataframes

In [ ]:
int_sandwich_df = sandwich_df[sandwich_df['int_ext'] == 'interior']
int_sandwich_df = int_sandwich_df.reset_index(drop=True)

In [ ]:
ext_sandwich_df = sandwich_df[sandwich_df['int_ext'] == 'exterior']
ext_sandwich_df = ext_sandwich_df.reset_index(drop=True)

In [ ]:
edg_int_sandwich_df = sandwich_df[  (sandwich_df['edge_or_central'] == 'edge')
                                  & (sandwich_df['int_ext'] == 'interior')]
edg_int_sandwich_df = edg_int_sandwich_df.reset_index(drop=True)

In [ ]:
cen_int_sandwich_df = sandwich_df[  (sandwich_df['edge_or_central'] == 'central')
                                  & (sandwich_df['int_ext'] == 'interior')]
cen_int_sandwich_df = cen_int_sandwich_df.reset_index(drop=True)

In [ ]:
edg_ext_sandwich_df = sandwich_df[  (sandwich_df['edge_or_central'] == 'edge')
                                  & (sandwich_df['int_ext'] == 'exterior')]
edg_ext_sandwich_df = edg_ext_sandwich_df.reset_index(drop=True)

In [ ]:
cen_ext_sandwich_df = sandwich_df[  (sandwich_df['edge_or_central'] == 'central')
                                  & (sandwich_df['int_ext'] == 'exterior')]
cen_ext_sandwich_df = cen_ext_sandwich_df.reset_index(drop=True)

### Phi psi discrete bin propensities

In [ ]:
discrete_phi_psi_cluster_coords, discrete_phi_psi_cluster_dataframe = calc_voronoi_points(
    sandwich_df, 'phi', 'psi', 100, True, 'Phi_psi_discrete_bins'
)

In [ ]:
vor = scipy.spatial.Voronoi(discrete_phi_psi_cluster_coords)

plt.clf()
fig = scipy.spatial.voronoi_plot_2d(vor, show_vertices=False)
fig.set_size_inches(10, 10)
plt.savefig('Phi_psi_discrete_bins_voronoi_points.svg')
plt.show()

In [ ]:
(discrete_phi_psi_propensity, discrete_phi_psi_frequency, discrete_phi_psi_normed_frequencies
) = calc_discrete_2d_indv_aa_propensities(sandwich_df, 'phi', 'psi', aa_dict, discrete_phi_psi_cluster_coords)

In [ ]:
discrete_phi_psi_propensity

In [ ]:
discrete_phi_psi_frequency

In [ ]:
plot_heat_map(discrete_phi_psi_propensity, True, 'Phi_psi_discrete_propensity')

In [ ]:
plot_heat_map(discrete_phi_psi_frequency, True, 'Phi_psi_discrete_frequency')

In [ ]:
plot_ramachandran_heat_map(
    discrete_phi_psi_propensity, discrete_phi_psi_cluster_coords, True, 'Ramachandran_propensity_plot'
)

In [ ]:
plot_ramachandran_heat_map(
    discrete_phi_psi_frequency, discrete_phi_psi_cluster_coords, True, 'Ramachandran_frequency_plot'
)

In [ ]:
(
    discrete_phi_psi_bootstrap_propensity_dict, discrete_phi_psi_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    sandwich_df, ['phi', 'psi'], aa_dict, [10, 30, 100, 300, 1000], calc_discrete_2d_indv_aa_propensities,
    discrete_phi_psi_propensity, discrete_phi_psi_frequency, cluster_coords=discrete_phi_psi_cluster_coords
)

In [ ]:
(
    discrete_phi_psi_conf_intv_propensity_dict, discrete_phi_psi_conf_intv_frequency_dict,
    discrete_phi_psi_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    sandwich_df, ['phi', 'psi'], aa_dict, 300, calc_discrete_2d_indv_aa_propensities, discrete_phi_psi_cluster_coords
)

In [ ]:
(
    lower_percentile_discrete_phi_psi_propensity_dict, upper_percentile_discrete_phi_psi_propensity_dict
) = gen_95_conf_intervals(
    discrete_phi_psi_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    discrete_phi_psi_propensity, lower_percentile_discrete_phi_psi_propensity_dict,
    upper_percentile_discrete_phi_psi_propensity_dict, 'Propensity', True, 'Phi_psi_discrete_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_discrete_phi_psi_frequency_dict, upper_percentile_discrete_phi_psi_frequency_dict
) = gen_95_conf_intervals(
    discrete_phi_psi_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    discrete_phi_psi_frequency, lower_percentile_discrete_phi_psi_frequency_dict,
    upper_percentile_discrete_phi_psi_frequency_dict, 'Frequency', True, 'Phi_psi_discrete_frequency_95_conf'
)

Interior surface

In [ ]:
discrete_phi_psi_int_cluster_coords, discrete_phi_psi_int_cluster_dataframe = calc_voronoi_points(
    int_sandwich_df, 'phi', 'psi', 100
)

In [ ]:
int_vor = scipy.spatial.Voronoi(discrete_phi_psi_int_cluster_coords)

plt.clf()
fig = scipy.spatial.voronoi_plot_2d(int_vor, show_vertices=False)
fig.set_size_inches(10, 10)
plt.savefig('Int_phi_psi_discrete_bins_voronoi_points.svg')
plt.show()

In [ ]:
(discrete_phi_psi_int_propensity, discrete_phi_psi_int_frequency, discrete_phi_psi_int_normed_frequencies
) = calc_discrete_2d_indv_aa_propensities(
    int_sandwich_df, 'phi', 'psi', aa_dict, discrete_phi_psi_int_cluster_coords
)

In [ ]:
discrete_phi_psi_int_propensity

In [ ]:
discrete_phi_psi_int_frequency

In [ ]:
plot_heat_map(discrete_phi_psi_int_propensity, True, 'Int_phi_psi_discrete_propensity')

In [ ]:
plot_heat_map(discrete_phi_psi_int_frequency, True, 'Int_phi_psi_discrete_frequency')

In [ ]:
plot_ramachandran_heat_map(
    discrete_phi_psi_int_propensity, discrete_phi_psi_int_cluster_coords, True, 'Int_Ramachandran_propensity_plot'
)

In [ ]:
plot_ramachandran_heat_map(
    discrete_phi_psi_int_frequency, discrete_phi_psi_int_cluster_coords, True, 'Int_Ramachandran_frequency_plot'
)

In [ ]:
(
    discrete_phi_psi_int_bootstrap_propensity_dict, discrete_phi_psi_int_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    int_sandwich_df, ['phi', 'psi'], aa_dict, [10, 30, 100, 300, 1000], calc_discrete_2d_indv_aa_propensities,
    discrete_phi_psi_int_propensity, discrete_phi_psi_int_frequency, discrete_phi_psi_int_cluster_coords
)

In [ ]:
(
    discrete_phi_psi_int_conf_intv_propensity_dict, discrete_phi_psi_int_conf_intv_frequency_dict,
    discrete_phi_psi_int_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    int_sandwich_df, ['phi', 'psi'], aa_dict, 300, calc_discrete_2d_indv_aa_propensities,
    discrete_phi_psi_int_cluster_coords
)

In [ ]:
(
    lower_percentile_discrete_phi_psi_int_propensity_dict, upper_percentile_discrete_phi_psi_int_propensity_dict
) = gen_95_conf_intervals(
    discrete_phi_psi_int_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    discrete_phi_psi_int_propensity, lower_percentile_discrete_phi_psi_int_propensity_dict,
    upper_percentile_discrete_phi_psi_int_propensity_dict, 'Propensity', True,
    'Int_phi_psi_discrete_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_discrete_phi_psi_int_frequency_dict, upper_percentile_discrete_phi_psi_int_frequency_dict
) = gen_95_conf_intervals(
    discrete_phi_psi_int_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    discrete_phi_psi_int_frequency, lower_percentile_discrete_phi_psi_int_frequency_dict,
    upper_percentile_discrete_phi_psi_int_frequency_dict, 'Frequency', True,
    'Int_phi_psi_discrete_frequency_95_conf'
)

Exterior surface

In [ ]:
discrete_phi_psi_ext_cluster_coords, discrete_phi_psi_ext_cluster_dataframe = calc_voronoi_points(
    ext_sandwich_df, 'phi', 'psi', 100
)

In [ ]:
ext_vor = scipy.spatial.Voronoi(discrete_phi_psi_ext_cluster_coords)

plt.clf()
fig = scipy.spatial.voronoi_plot_2d(ext_vor, show_vertices=False)
fig.set_size_inches(10, 10)
plt.savefig('Ext_phi_psi_discrete_bins_voronoi_points.svg')
plt.show()

In [ ]:
(
    discrete_phi_psi_ext_propensity, discrete_phi_psi_ext_frequency, discrete_phi_psi_ext_normed_frequencies
) = calc_discrete_2d_indv_aa_propensities(
    ext_sandwich_df, 'phi', 'psi', aa_dict, discrete_phi_psi_ext_cluster_coords
)

In [ ]:
discrete_phi_psi_ext_propensity

In [ ]:
discrete_phi_psi_ext_frequency

In [ ]:
plot_heat_map(discrete_phi_psi_ext_propensity, True, 'Ext_phi_psi_discrete_propensity')

In [ ]:
plot_heat_map(discrete_phi_psi_ext_frequency, True, 'Ext_phi_psi_discrete_frequency')

In [ ]:
plot_ramachandran_heat_map(
    discrete_phi_psi_ext_propensity, discrete_phi_psi_ext_cluster_coords, True, 'Ext_Ramachandran_propensity_plot'
)

In [ ]:
plot_ramachandran_heat_map(
    discrete_phi_psi_ext_frequency, discrete_phi_psi_ext_cluster_coords, True, 'Ext_Ramachandran_frequency_plot'
)

In [ ]:
(
    discrete_phi_psi_ext_bootstrap_propensity_dict, discrete_phi_psi_ext_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    ext_sandwich_df, ['phi', 'psi'], aa_dict, [10, 30, 100, 300, 1000], calc_discrete_2d_indv_aa_propensities,
    discrete_phi_psi_ext_propensity, discrete_phi_psi_ext_frequency, discrete_phi_psi_ext_cluster_coords
)

In [ ]:
(
    discrete_phi_psi_ext_conf_intv_propensity_dict, discrete_phi_psi_ext_conf_intv_frequency_dict,
    discrete_phi_psi_ext_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    ext_sandwich_df, ['phi', 'psi'], aa_dict, 300, calc_discrete_2d_indv_aa_propensities,
    discrete_phi_psi_ext_cluster_coords
)

In [ ]:
(
    lower_percentile_discrete_phi_psi_ext_propensity_dict, upper_percentile_discrete_phi_psi_ext_propensity_dict
) = gen_95_conf_intervals(
    discrete_phi_psi_ext_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    discrete_phi_psi_ext_propensity, lower_percentile_discrete_phi_psi_ext_propensity_dict,
    upper_percentile_discrete_phi_psi_ext_propensity_dict, 'Propensity', True,
    'Ext_phi_psi_discrete_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_discrete_phi_psi_ext_frequency_dict, upper_percentile_discrete_phi_psi_ext_frequency_dict
) = gen_95_conf_intervals(
    discrete_phi_psi_ext_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    discrete_phi_psi_ext_frequency, lower_percentile_discrete_phi_psi_ext_frequency_dict,
    upper_percentile_discrete_phi_psi_ext_frequency_dict, 'Frequency', True,
    'Ext_phi_psi_discrete_frequency_95_conf'
)

### Buried surface area discrete bin propensities

In [ ]:
boolean_bsa_list = ['']*sandwich_df.shape[0]
for row in range(sandwich_df.shape[0]):
    bsa = sandwich_df['buried_surface_area'][row]
    try:
        bsa = float(bsa)
        if bsa == 0:
            boolean_bsa_list[row] = 'No change'
        elif bsa > 0:
            boolean_bsa_list[row] = 'decrease'
    except ValueError:
        pass

bsa_df = copy.deepcopy(sandwich_df)
bsa_df['buried_surface_area'] = boolean_bsa_list

bsa_disc_propensity, bsa_disc_frequency, bsa_disc_normed_frequencies = calc_indv_property_propensities(
    bsa_df, 'buried_surface_area', aa_dict
)

In [ ]:
bsa_disc_propensity

In [ ]:
bsa_disc_frequency

In [ ]:
plot_bar_graphs(bsa_disc_propensity, 'Propensity', True, 'BSA_discrete_propensity')

In [ ]:
plot_bar_graphs(bsa_disc_frequency, 'Frequency', True, 'BSA_discrete_frequency')

In [ ]:
plot_heat_map(bsa_disc_propensity)

In [ ]:
plot_heat_map(bsa_disc_frequency)

In [ ]:
(
    bsa_disc_bootstrap_propensity_dict, bsa_disc_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    bsa_df, 'buried_surface_area', aa_dict, [10, 30, 100, 300, 1000], calc_indv_property_propensities,
    bsa_disc_propensity, bsa_disc_frequency
)

In [ ]:
(
    bsa_disc_conf_intv_propensity_dict, bsa_disc_conf_intv_frequency_dict, bsa_disc_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    bsa_df, 'buried_surface_area', aa_dict, 300, calc_indv_property_propensities
)

In [ ]:
lower_percentile_bsa_disc_propensity_dict, upper_percentile_bsa_disc_propensity_dict = gen_95_conf_intervals(
    bsa_disc_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    bsa_disc_propensity, lower_percentile_bsa_disc_propensity_dict, upper_percentile_bsa_disc_propensity_dict,
    'Propensity', True, 'BSA_discrete_propensity_95_conf'
)

In [ ]:
lower_percentile_bsa_disc_frequency_dict, upper_percentile_bsa_disc_frequency_dict = gen_95_conf_intervals(
    bsa_disc_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    bsa_disc_frequency, lower_percentile_bsa_disc_frequency_dict, upper_percentile_bsa_disc_frequency_dict,
    'Frequency', True, 'BSA_discrete_frequency_95_conf'
)

### Amino acid pairs propensities

In [ ]:
os.chdir('..')
if not os.path.isdir('Discrete_propensities_pairs_aas'):
    os.mkdir('Discrete_propensities_pairs_aas')
os.chdir('Discrete_propensities_pairs_aas')

#### Hydrogen bonding pairs

All sandwich surfaces

In [ ]:
hb_aa_pairs = gen_neighbouring_pairs_list(sandwich_df, 'hb_pairs', aa_dict)
(hb_aa_pair_propensity, hb_aa_pair_frequency, hb_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(hb_aa_pairs, aa_dict)

In [ ]:
hb_aa_pair_propensity

In [ ]:
hb_aa_pair_frequency

In [ ]:
plot_heat_map(hb_aa_pair_propensity, True, 'HB_pairs_propensity')

In [ ]:
plot_heat_map(hb_aa_pair_frequency, True, 'HB_pairs_frequency')

In [ ]:
(
    hb_aa_pair_bootstrap_propensity_dict, hb_aa_pair_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    sandwich_df, 'hb_pairs', aa_dict, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    hb_aa_pair_propensity, hb_aa_pair_frequency
)

In [ ]:
(
    hb_aa_pair_conf_intv_propensity_dict, hb_aa_pair_conf_intv_frequency_dict,
    hb_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    sandwich_df, 'hb_pairs', aa_dict, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_hb_aa_pair_propensity_dict, upper_percentile_hb_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    hb_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    hb_aa_pair_propensity, lower_percentile_hb_aa_pair_propensity_dict,
    upper_percentile_hb_aa_pair_propensity_dict, 'Propensity', True, 'HB_pairs_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_hb_aa_pair_frequency_dict, upper_percentile_hb_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    hb_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    hb_aa_pair_frequency, lower_percentile_hb_aa_pair_frequency_dict,
    upper_percentile_hb_aa_pair_frequency_dict, 'Frequency', True, 'HB_pairs_frequency_95_conf'
)

Interior surface

In [ ]:
hb_int_aa_pairs = gen_neighbouring_pairs_list(int_sandwich_df, 'hb_pairs', aa_dict)
(hb_int_aa_pair_propensity, hb_int_aa_pair_frequency, hb_int_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(hb_int_aa_pairs, aa_dict)

In [ ]:
hb_int_aa_pair_propensity

In [ ]:
hb_int_aa_pair_frequency

In [ ]:
plot_heat_map(hb_int_aa_pair_propensity, True, 'Int_HB_pairs_propensity')

In [ ]:
plot_heat_map(hb_int_aa_pair_frequency, True, 'Int_HB_pairs_frequency')

In [ ]:
(
    hb_int_aa_pair_bootstrap_propensity_dict, hb_int_aa_pair_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    int_sandwich_df, 'hb_pairs', aa_dict, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    hb_int_aa_pair_propensity, hb_int_aa_pair_frequency
)

In [ ]:
(
    hb_int_aa_pair_conf_intv_propensity_dict, hb_int_aa_pair_conf_intv_frequency_dict,
    hb_int_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    int_sandwich_df, 'hb_pairs', aa_dict, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_hb_int_aa_pair_propensity_dict, upper_percentile_hb_int_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    hb_int_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    hb_int_aa_pair_propensity, lower_percentile_hb_int_aa_pair_propensity_dict,
    upper_percentile_hb_int_aa_pair_propensity_dict, 'Propensity', True, 'Int_HB_pairs_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_hb_int_aa_pair_frequency_dict, upper_percentile_hb_int_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    hb_int_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    hb_int_aa_pair_frequency, lower_percentile_hb_int_aa_pair_frequency_dict,
    upper_percentile_hb_int_aa_pair_frequency_dict, 'Frequency', True, 'Int_HB_pairs_frequency_95_conf'
)

Exterior surface

In [ ]:
hb_ext_aa_pairs = gen_neighbouring_pairs_list(ext_sandwich_df, 'hb_pairs', aa_dict)
(hb_ext_aa_pair_propensity, hb_ext_aa_pair_frequency, hb_ext_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(hb_ext_aa_pairs, aa_dict)

In [ ]:
hb_ext_aa_pair_propensity

In [ ]:
hb_ext_aa_pair_frequency

In [ ]:
plot_heat_map(hb_ext_aa_pair_propensity, True, 'Ext_HB_pairs_propensity')

In [ ]:
plot_heat_map(hb_ext_aa_pair_frequency, True, 'Ext_HB_pairs_frequency')

In [ ]:
(
    hb_ext_aa_pair_bootstrap_propensity_dict, hb_ext_aa_pair_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    ext_sandwich_df, 'hb_pairs', aa_dict, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    hb_ext_aa_pair_propensity, hb_ext_aa_pair_frequency
)

In [ ]:
(
    hb_ext_aa_pair_conf_intv_propensity_dict, hb_ext_aa_pair_conf_intv_frequency_dict,
    hb_ext_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    ext_sandwich_df, 'hb_pairs', aa_dict, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_hb_ext_aa_pair_propensity_dict, upper_percentile_hb_ext_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    hb_ext_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    hb_ext_aa_pair_propensity, lower_percentile_hb_ext_aa_pair_propensity_dict,
    upper_percentile_hb_ext_aa_pair_propensity_dict, 'Propensity', True, 'Ext_HB_pairs_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_hb_ext_aa_pair_frequency_dict, upper_percentile_hb_ext_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    hb_ext_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    hb_ext_aa_pair_frequency, lower_percentile_hb_ext_aa_pair_frequency_dict,
    upper_percentile_hb_ext_aa_pair_frequency_dict, 'Frequency', True, 'Ext_HB_pairs_frequency_95_conf'
)

#### Non-hydrogen bonding pairs

All sandwich surfaces

In [ ]:
nhb_aa_pairs = gen_neighbouring_pairs_list(sandwich_df, 'nhb_pairs', aa_dict)
(nhb_aa_pair_propensity, nhb_aa_pair_frequency, nhb_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(nhb_aa_pairs, aa_dict)

In [ ]:
nhb_aa_pair_propensity

In [ ]:
nhb_aa_pair_frequency

In [ ]:
plot_heat_map(nhb_aa_pair_propensity, True, 'NHB_pairs_propensity')

In [ ]:
plot_heat_map(nhb_aa_pair_frequency, True, 'NHB_pairs_frequency')

In [ ]:
(
    nhb_aa_pair_bootstrap_propensity_dict, nhb_aa_pair_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    sandwich_df, 'nhb_pairs', aa_dict, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    nhb_aa_pair_propensity, nhb_aa_pair_frequency
)

In [ ]:
(
    nhb_aa_pair_conf_intv_propensity_dict, nhb_aa_pair_conf_intv_frequency_dict,
    nhb_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    sandwich_df, 'nhb_pairs', aa_dict, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_nhb_aa_pair_propensity_dict, upper_percentile_nhb_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    nhb_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    nhb_aa_pair_propensity, lower_percentile_nhb_aa_pair_propensity_dict,
    upper_percentile_nhb_aa_pair_propensity_dict, 'Propensity', True, 'NHB_pairs_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_nhb_aa_pair_frequency_dict, upper_percentile_nhb_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    nhb_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    nhb_aa_pair_frequency, lower_percentile_nhb_aa_pair_frequency_dict,
    upper_percentile_nhb_aa_pair_frequency_dict, 'Frequency', True, 'NHB_pairs_frequency_95_conf'
)

Interior surface

In [ ]:
nhb_int_aa_pairs = gen_neighbouring_pairs_list(int_sandwich_df, 'nhb_pairs', aa_dict)
(nhb_int_aa_pair_propensity, nhb_int_aa_pair_frequency, nhb_int_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(nhb_int_aa_pairs, aa_dict)

In [ ]:
nhb_int_aa_pair_propensity

In [ ]:
nhb_int_aa_pair_frequency

In [ ]:
plot_heat_map(nhb_int_aa_pair_propensity, True, 'Int_NHB_pairs_propensity')

In [ ]:
plot_heat_map(nhb_int_aa_pair_frequency, True, 'Int_NHB_pairs_frequency')

In [ ]:
(
    nhb_int_aa_pair_bootstrap_propensity_dict, nhb_int_aa_pair_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    int_sandwich_df, 'nhb_pairs', aa_dict, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    nhb_int_aa_pair_propensity, nhb_int_aa_pair_frequency
)

In [ ]:
(
    nhb_int_aa_pair_conf_intv_propensity_dict, nhb_int_aa_pair_conf_intv_frequency_dict,
    nhb_int_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    int_sandwich_df, 'nhb_pairs', aa_dict, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_nhb_int_aa_pair_propensity_dict, upper_percentile_nhb_int_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    nhb_int_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    nhb_int_aa_pair_propensity, lower_percentile_nhb_int_aa_pair_propensity_dict,
    upper_percentile_nhb_int_aa_pair_propensity_dict, 'Propensity', True, 'Int_NHB_pairs_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_nhb_int_aa_pair_frequency_dict, upper_percentile_nhb_int_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    nhb_int_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    nhb_int_aa_pair_frequency, lower_percentile_nhb_int_aa_pair_frequency_dict,
    upper_percentile_nhb_int_aa_pair_frequency_dict, 'Frequency', True, 'Int_NHB_pairs_frequency_95_conf'
)

Exterior surface

In [ ]:
nhb_ext_aa_pairs = gen_neighbouring_pairs_list(ext_sandwich_df, 'nhb_pairs', aa_dict)
(nhb_ext_aa_pair_propensity, nhb_ext_aa_pair_frequency, nhb_ext_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(nhb_ext_aa_pairs, aa_dict)

In [ ]:
nhb_ext_aa_pair_propensity

In [ ]:
nhb_ext_aa_pair_frequency

In [ ]:
plot_heat_map(nhb_ext_aa_pair_propensity, True, 'Ext_NHB_pairs_propensity')

In [ ]:
plot_heat_map(nhb_ext_aa_pair_frequency, True, 'Ext_NHB_pairs_frequency')

In [ ]:
(
    nhb_ext_aa_pair_bootstrap_propensity_dict, nhb_ext_aa_pair_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    ext_sandwich_df, 'nhb_pairs', aa_dict, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    nhb_ext_aa_pair_propensity, nhb_ext_aa_pair_frequency
)

In [ ]:
(
    nhb_ext_aa_pair_conf_intv_propensity_dict, nhb_ext_aa_pair_conf_intv_frequency_dict,
    nhb_ext_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    ext_sandwich_df, 'nhb_pairs', aa_dict, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_nhb_ext_aa_pair_propensity_dict, upper_percentile_nhb_ext_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    nhb_ext_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    nhb_ext_aa_pair_propensity, lower_percentile_nhb_ext_aa_pair_propensity_dict,
    upper_percentile_nhb_ext_aa_pair_propensity_dict, 'Propensity', True, 'Ext_NHB_pairs_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_nhb_ext_aa_pair_frequency_dict, upper_percentile_nhb_ext_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    nhb_ext_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    nhb_ext_aa_pair_frequency, lower_percentile_nhb_ext_aa_pair_frequency_dict,
    upper_percentile_nhb_ext_aa_pair_frequency_dict, 'Frequency', True, 'Ext_NHB_pairs_frequency_95_conf'
)

#### Minus 2 pairs

All sandwich surfaces

In [ ]:
minus2_aa_pairs = gen_neighbouring_pairs_list(sandwich_df, 'minus_2', aa_dict)
(minus2_aa_pair_propensity, minus2_aa_pair_frequency, minus2_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(minus2_aa_pairs, aa_dict)

In [ ]:
minus2_aa_pair_propensity

In [ ]:
minus2_aa_pair_frequency

In [ ]:
plot_heat_map(minus2_aa_pair_propensity, True, 'Minus2_pairs_propensity')

In [ ]:
plot_heat_map(minus2_aa_pair_frequency, True, 'Minus2_pairs_frequency')

In [ ]:
(
    minus2_aa_pair_bootstrap_propensity_dict, minus2_aa_pair_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    sandwich_df, 'minus_2', aa_dict, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    minus2_aa_pair_propensity, minus2_aa_pair_frequency
)

In [ ]:
(
    minus2_aa_pair_conf_intv_propensity_dict, minus2_aa_pair_conf_intv_frequency_dict,
    minus2_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    sandwich_df, 'minus_2', aa_dict, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_minus2_aa_pair_propensity_dict, upper_percentile_minus2_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    minus2_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    minus2_aa_pair_propensity, lower_percentile_minus2_aa_pair_propensity_dict,
    upper_percentile_minus2_aa_pair_propensity_dict, 'Propensity', True, 'Minus2_pairs_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_minus2_aa_pair_frequency_dict, upper_percentile_minus2_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    minus2_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    minus2_aa_pair_frequency, lower_percentile_minus2_aa_pair_frequency_dict,
    upper_percentile_minus2_aa_pair_frequency_dict, 'Frequency', True, 'Minus2_pairs_frequency_95_conf'
)

Interior surface

In [ ]:
minus2_int_aa_pairs = gen_neighbouring_pairs_list(int_sandwich_df, 'minus_2', aa_dict)
(minus2_int_aa_pair_propensity, minus2_int_aa_pair_frequency, minus2_int_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(minus2_int_aa_pairs, aa_dict)

In [ ]:
minus2_int_aa_pair_propensity

In [ ]:
minus2_int_aa_pair_frequency

In [ ]:
plot_heat_map(minus2_int_aa_pair_propensity, True, 'Int_Minus2_pairs_propensity')

In [ ]:
plot_heat_map(minus2_int_aa_pair_frequency, True, 'Int_Minus2_pairs_frequency')

In [ ]:
(
    minus2_int_aa_pair_bootstrap_propensity_dict, minus2_int_aa_pair_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    int_sandwich_df, 'minus_2', aa_dict, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    minus2_int_aa_pair_propensity, minus2_int_aa_pair_frequency
)

In [ ]:
(
    minus2_int_aa_pair_conf_intv_propensity_dict, minus2_int_aa_pair_conf_intv_frequency_dict,
    minus2_int_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    int_sandwich_df, 'minus_2', aa_dict, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_minus2_int_aa_pair_propensity_dict, upper_percentile_minus2_int_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    minus2_int_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    minus2_int_aa_pair_propensity, lower_percentile_minus2_int_aa_pair_propensity_dict,
    upper_percentile_minus2_int_aa_pair_propensity_dict, 'Propensity', True, 'Int_Minus2_pairs_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_minus2_int_aa_pair_frequency_dict, upper_percentile_minus2_int_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    minus2_int_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    minus2_int_aa_pair_frequency, lower_percentile_minus2_int_aa_pair_frequency_dict,
    upper_percentile_minus2_int_aa_pair_frequency_dict, 'Frequency', True, 'Int_Minus2_pairs_frequency_95_conf'
)

Exterior surface

In [ ]:
minus2_ext_aa_pairs = gen_neighbouring_pairs_list(ext_sandwich_df, 'minus_2', aa_dict)
(minus2_ext_aa_pair_propensity, minus2_ext_aa_pair_frequency, minus2_ext_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(minus2_ext_aa_pairs, aa_dict)

In [ ]:
minus2_ext_aa_pair_propensity

In [ ]:
minus2_ext_aa_pair_frequency

In [ ]:
plot_heat_map(minus2_ext_aa_pair_propensity, True, 'Ext_Minus2_pairs_propensity')

In [ ]:
plot_heat_map(minus2_ext_aa_pair_frequency, True, 'Ext_Minus2_pairs_frequency')

In [ ]:
(
    minus2_ext_aa_pair_bootstrap_propensity_dict, minus2_ext_aa_pair_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    ext_sandwich_df, 'minus_2', aa_dict, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    minus2_ext_aa_pair_propensity, minus2_ext_aa_pair_frequency
)

In [ ]:
(
    minus2_ext_aa_pair_conf_intv_propensity_dict, minus2_ext_aa_pair_conf_intv_frequency_dict,
    minus2_ext_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    ext_sandwich_df, 'minus_2', aa_dict, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_minus2_ext_aa_pair_propensity_dict, upper_percentile_minus2_ext_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    minus2_ext_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    minus2_ext_aa_pair_propensity, lower_percentile_minus2_ext_aa_pair_propensity_dict,
    upper_percentile_minus2_ext_aa_pair_propensity_dict, 'Propensity', True, 'Ext_Minus2_pairs_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_minus2_ext_aa_pair_frequency_dict, upper_percentile_minus2_ext_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    minus2_ext_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    minus2_ext_aa_pair_frequency, lower_percentile_minus2_ext_aa_pair_frequency_dict,
    upper_percentile_minus2_ext_aa_pair_frequency_dict, 'Frequency', True, 'Ext_Minus2_pairs_frequency_95_conf'
)

#### Plus 2 pairs

All sandwich surfaces

In [ ]:
plus2_aa_pairs = gen_neighbouring_pairs_list(sandwich_df, 'plus_2', aa_dict)
(plus2_aa_pair_propensity, plus2_aa_pair_frequency, plus2_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(plus2_aa_pairs, aa_dict)

In [ ]:
plus2_aa_pair_propensity

In [ ]:
plus2_aa_pair_frequency

In [ ]:
plot_heat_map(plus2_aa_pair_propensity, True, 'Plus2_pairs_propensity')

In [ ]:
plot_heat_map(plus2_aa_pair_frequency, True, 'Plus2_pairs_frequency')

In [ ]:
(
    plus2_aa_pair_bootstrap_propensity_dict, plus2_aa_pair_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    sandwich_df, 'plus_2', aa_dict, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    plus2_aa_pair_propensity, plus2_aa_pair_frequency
)

In [ ]:
(
    plus2_aa_pair_conf_intv_propensity_dict, plus2_aa_pair_conf_intv_frequency_dict,
    plus2_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    sandwich_df, 'plus_2', aa_dict, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_plus2_aa_pair_propensity_dict, upper_percentile_plus2_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    plus2_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    plus2_aa_pair_propensity, lower_percentile_plus2_aa_pair_propensity_dict,
    upper_percentile_plus2_aa_pair_propensity_dict, 'Propensity', True, 'Plus2_pairs_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_plus2_aa_pair_frequency_dict, upper_percentile_plus2_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    plus2_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    plus2_aa_pair_frequency, lower_percentile_plus2_aa_pair_frequency_dict,
    upper_percentile_plus2_aa_pair_frequency_dict, 'Frequency', True, 'Plus2_pairs_frequency_95_conf'
)

Interior surface

In [ ]:
plus2_int_aa_pairs = gen_neighbouring_pairs_list(int_sandwich_df, 'plus_2', aa_dict)
(plus2_int_aa_pair_propensity, plus2_int_aa_pair_frequency, plus2_int_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(plus2_int_aa_pairs, aa_dict)

In [ ]:
plus2_int_aa_pair_propensity

In [ ]:
plus2_int_aa_pair_frequency

In [ ]:
plot_heat_map(plus2_int_aa_pair_propensity, True, 'Int_Plus2_pairs_propensity')

In [ ]:
plot_heat_map(plus2_int_aa_pair_frequency, True, 'Int_Plus2_pairs_frequency')

In [ ]:
(
    plus2_int_aa_pair_bootstrap_propensity_dict, plus2_int_aa_pair_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    int_sandwich_df, 'plus_2', aa_dict, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    plus2_int_aa_pair_propensity, plus2_int_aa_pair_frequency
)

In [ ]:
(
    plus2_int_aa_pair_conf_intv_propensity_dict, plus2_int_aa_pair_conf_intv_frequency_dict,
    plus2_int_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    int_sandwich_df, 'plus_2', aa_dict, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_plus2_int_aa_pair_propensity_dict, upper_percentile_plus2_int_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    plus2_int_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    plus2_int_aa_pair_propensity, lower_percentile_plus2_int_aa_pair_propensity_dict,
    upper_percentile_plus2_int_aa_pair_propensity_dict, 'Propensity', True, 'Int_Plus2_pairs_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_plus2_int_aa_pair_frequency_dict, upper_percentile_plus2_int_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    plus2_int_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    plus2_int_aa_pair_frequency, lower_percentile_plus2_int_aa_pair_frequency_dict,
    upper_percentile_plus2_int_aa_pair_frequency_dict, 'Frequency', True, 'Int_Plus2_pairs_frequency_95_conf'
)

Exterior surface

In [ ]:
plus2_ext_aa_pairs = gen_neighbouring_pairs_list(ext_sandwich_df, 'plus_2', aa_dict)
(plus2_ext_aa_pair_propensity, plus2_ext_aa_pair_frequency, plus2_ext_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(plus2_ext_aa_pairs, aa_dict)

In [ ]:
plus2_ext_aa_pair_propensity

In [ ]:
plus2_ext_aa_pair_frequency

In [ ]:
plot_heat_map(plus2_ext_aa_pair_propensity, True, 'Ext_Plus2_pairs_propensity')

In [ ]:
plot_heat_map(plus2_ext_aa_pair_frequency, True, 'Ext_Plus2_pairs_frequency')

In [ ]:
(
    plus2_ext_aa_pair_bootstrap_propensity_dict, plus2_ext_aa_pair_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    ext_sandwich_df, 'plus_2', aa_dict, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    plus2_ext_aa_pair_propensity, plus2_ext_aa_pair_frequency
)

In [ ]:
(
    plus2_ext_aa_pair_conf_intv_propensity_dict, plus2_ext_aa_pair_conf_intv_frequency_dict,
    plus2_ext_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    ext_sandwich_df, 'plus_2', aa_dict, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_plus2_ext_aa_pair_propensity_dict, upper_percentile_plus2_ext_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    plus2_ext_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    plus2_ext_aa_pair_propensity, lower_percentile_plus2_ext_aa_pair_propensity_dict,
    upper_percentile_plus2_ext_aa_pair_propensity_dict, 'Propensity', True, 'Ext_Plus2_pairs_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_plus2_ext_aa_pair_frequency_dict, upper_percentile_plus2_ext_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    plus2_ext_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    plus2_ext_aa_pair_frequency, lower_percentile_plus2_ext_aa_pair_frequency_dict,
    upper_percentile_plus2_ext_aa_pair_frequency_dict, 'Frequency', True, 'Ext_Plus2_pairs_frequency_95_conf'
)

#### Minus 1 pairs

All sandwich surfaces

In [ ]:
minus1_aa_pairs = gen_neighbouring_pairs_list(sandwich_df, 'minus_1', aa_dict)
(minus1_aa_pair_propensity, minus1_aa_pair_frequency, minus1_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(minus1_aa_pairs, aa_dict)

In [ ]:
minus1_aa_pair_propensity

In [ ]:
minus1_aa_pair_frequency

In [ ]:
plot_heat_map(minus1_aa_pair_propensity, True, 'Minus1_pairs_propensity')

In [ ]:
plot_heat_map(minus1_aa_pair_frequency, True, 'Minus1_pairs_frequency')

In [ ]:
(
    minus1_aa_pair_bootstrap_propensity_dict, minus1_aa_pair_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    sandwich_df, 'minus_1', aa_dict, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    minus1_aa_pair_propensity, minus1_aa_pair_frequency
)

In [ ]:
(
    minus1_aa_pair_conf_intv_propensity_dict, minus1_aa_pair_conf_intv_frequency_dict,
    minus1_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    sandwich_df, 'minus_1', aa_dict, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_minus1_aa_pair_propensity_dict, upper_percentile_minus1_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    minus1_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    minus1_aa_pair_propensity, lower_percentile_minus1_aa_pair_propensity_dict,
    upper_percentile_minus1_aa_pair_propensity_dict, 'Propensity', True, 'Minus1_pairs_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_minus1_aa_pair_frequency_dict, upper_percentile_minus1_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    minus1_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    minus1_aa_pair_frequency, lower_percentile_minus1_aa_pair_frequency_dict,
    upper_percentile_minus1_aa_pair_frequency_dict, 'Frequency', True, 'Minus1_pairs_freqency_95_conf'
)

Interior surface

In [ ]:
minus1_int_aa_pairs = gen_neighbouring_pairs_list(int_sandwich_df, 'minus_1', aa_dict)
(minus1_int_aa_pair_propensity, minus1_int_aa_pair_frequency, minus1_int_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(minus1_int_aa_pairs, aa_dict)

In [ ]:
minus1_int_aa_pair_propensity

In [ ]:
minus1_int_aa_pair_frequency

In [ ]:
plot_heat_map(minus1_int_aa_pair_propensity, True, 'Int_minus1_pairs_propensity')

In [ ]:
plot_heat_map(minus1_int_aa_pair_frequency, True, 'Int_minus1_pairs_frequency')

In [ ]:
(
    minus1_int_aa_pair_bootstrap_propensity_dict, minus1_int_aa_pair_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    int_sandwich_df, 'minus_1', aa_dict, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    minus1_int_aa_pair_propensity, minus1_int_aa_pair_frequency
)

In [ ]:
(
    minus1_int_aa_pair_conf_intv_propensity_dict, minus1_int_aa_pair_conf_intv_frequency_dict,
    minus1_int_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    int_sandwich_df, 'minus_1', aa_dict, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_minus1_int_aa_pair_propensity_dict, upper_percentile_minus1_int_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    minus1_int_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    minus1_int_aa_pair_propensity, lower_percentile_minus1_int_aa_pair_propensity_dict,
    upper_percentile_minus1_int_aa_pair_propensity_dict, 'Propensity', True, 'Int_minus1_pairs_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_minus1_int_aa_pair_frequency_dict, upper_percentile_minus1_int_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    minus1_int_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    minus1_int_aa_pair_frequency, lower_percentile_minus1_int_aa_pair_frequency_dict,
    upper_percentile_minus1_int_aa_pair_frequency_dict, 'Frequency', True, 'Int_minus1_pairs_freqency_95_conf'
)

Exterior surface

In [ ]:
minus1_ext_aa_pairs = gen_neighbouring_pairs_list(ext_sandwich_df, 'minus_1', aa_dict)
(minus1_ext_aa_pair_propensity, minus1_ext_aa_pair_frequency, minus1_ext_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(minus1_ext_aa_pairs, aa_dict)

In [ ]:
minus1_ext_aa_pair_propensity

In [ ]:
minus1_ext_aa_pair_frequency

In [ ]:
plot_heat_map(minus1_ext_aa_pair_propensity, True, 'Ext_minus1_pairs_propensity')

In [ ]:
plot_heat_map(minus1_ext_aa_pair_frequency, True, 'Ext_minus1_pairs_frequency')

In [ ]:
(
    minus1_ext_aa_pair_bootstrap_propensity_dict, minus1_ext_aa_pair_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    ext_sandwich_df, 'minus_1', aa_dict, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    minus1_ext_aa_pair_propensity, minus1_ext_aa_pair_frequency
)

In [ ]:
(
    minus1_ext_aa_pair_conf_intv_propensity_dict, minus1_ext_aa_pair_conf_intv_frequency_dict,
    minus1_ext_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    ext_sandwich_df, 'minus_1', aa_dict, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_minus1_ext_aa_pair_propensity_dict, upper_percentile_minus1_ext_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    minus1_ext_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    minus1_ext_aa_pair_propensity, lower_percentile_minus1_ext_aa_pair_propensity_dict,
    upper_percentile_minus1_ext_aa_pair_propensity_dict, 'Propensity', True, 'Ext_minus1_pairs_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_minus1_ext_aa_pair_frequency_dict, upper_percentile_minus1_ext_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    minus1_ext_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    minus1_ext_aa_pair_frequency, lower_percentile_minus1_ext_aa_pair_frequency_dict,
    upper_percentile_minus1_ext_aa_pair_frequency_dict, 'Frequency', True, 'Ext_minus1_pairs_frequency_95_conf'
)

#### Plus 1 pairs

All sandwich surfaces

In [ ]:
plus1_aa_pairs = gen_neighbouring_pairs_list(sandwich_df, 'plus_1', aa_dict)
(plus1_aa_pair_propensity, plus1_aa_pair_frequency, plus1_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(plus1_aa_pairs, aa_dict)

In [ ]:
plus1_aa_pair_propensity

In [ ]:
plus1_aa_pair_frequency

In [ ]:
plot_heat_map(plus1_aa_pair_propensity, True, 'Plus1_pairs_propensity')

In [ ]:
plot_heat_map(plus1_aa_pair_frequency, True, 'Plus1_pairs_frequency')

In [ ]:
(
    plus1_aa_pair_bootstrap_propensity_dict, plus1_aa_pair_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    sandwich_df, 'plus_1', aa_dict, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    plus1_aa_pair_propensity, plus1_aa_pair_frequency
)

In [ ]:
(
    plus1_aa_pair_conf_intv_propensity_dict, plus1_aa_pair_conf_intv_frequency_dict,
    plus1_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    sandwich_df, 'plus_1', aa_dict, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_plus1_aa_pair_propensity_dict, upper_percentile_plus1_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    plus1_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    plus1_aa_pair_propensity, lower_percentile_plus1_aa_pair_propensity_dict,
    upper_percentile_plus1_aa_pair_propensity_dict, 'Propensity', True, 'Plus1_pairs_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_plus1_aa_pair_frequency_dict, upper_percentile_plus1_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    plus1_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    plus1_aa_pair_frequency, lower_percentile_plus1_aa_pair_frequency_dict,
    upper_percentile_plus1_aa_pair_frequency_dict, 'Frequency', True, 'Plus1_pairs_frequency_95_conf'
)

Interior surface

In [ ]:
plus1_int_aa_pairs = gen_neighbouring_pairs_list(int_sandwich_df, 'plus_1', aa_dict)
(plus1_int_aa_pair_propensity, plus1_int_aa_pair_frequency, plus1_int_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(plus1_int_aa_pairs, aa_dict)

In [ ]:
plus1_int_aa_pair_propensity

In [ ]:
plus1_int_aa_pair_frequency

In [ ]:
plot_heat_map(plus1_int_aa_pair_propensity, True, 'Int_Plus1_pairs_propensity')

In [ ]:
plot_heat_map(plus1_int_aa_pair_frequency, True, 'Int_Plus1_pairs_frequency')

In [ ]:
(
    plus1_int_aa_pair_bootstrap_propensity_dict, plus1_int_aa_pair_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    int_sandwich_df, 'plus_1', aa_dict, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    plus1_int_aa_pair_propensity, plus1_int_aa_pair_frequency
)

In [ ]:
(
    plus1_int_aa_pair_conf_intv_propensity_dict, plus1_int_aa_pair_conf_intv_frequency_dict,
    plus1_int_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    int_sandwich_df, 'plus_1', aa_dict, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_plus1_int_aa_pair_propensity_dict, upper_percentile_plus1_int_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    plus1_int_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    plus1_int_aa_pair_propensity, lower_percentile_plus1_int_aa_pair_propensity_dict,
    upper_percentile_plus1_int_aa_pair_propensity_dict, 'Propensity', True, 'Int_Plus1_pairs_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_plus1_int_aa_pair_frequency_dict, upper_percentile_plus1_int_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    plus1_int_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    plus1_int_aa_pair_frequency, lower_percentile_plus1_int_aa_pair_frequency_dict,
    upper_percentile_plus1_int_aa_pair_frequency_dict, 'Frequency', True, 'Int_Plus1_pairs_frequency_95_conf'
)

Exterior surface

In [ ]:
plus1_ext_aa_pairs = gen_neighbouring_pairs_list(ext_sandwich_df, 'plus_1', aa_dict)
(plus1_ext_aa_pair_propensity, plus1_ext_aa_pair_frequency, plus1_ext_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(plus1_ext_aa_pairs, aa_dict)

In [ ]:
plus1_ext_aa_pair_propensity

In [ ]:
plus1_ext_aa_pair_frequency

In [ ]:
plot_heat_map(plus1_ext_aa_pair_propensity, True, 'Ext_Plus1_pairs_propensity')

In [ ]:
plot_heat_map(plus1_ext_aa_pair_frequency, True, 'Ext_Plus1_pairs_frequency')

In [ ]:
(
    plus1_ext_aa_pair_bootstrap_propensity_dict, plus1_ext_aa_pair_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    ext_sandwich_df, 'plus_1', aa_dict, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    plus1_ext_aa_pair_propensity, plus1_ext_aa_pair_frequency
)

In [ ]:
(
    plus1_ext_aa_pair_conf_intv_propensity_dict, plus1_ext_aa_pair_conf_intv_frequency_dict,
    plus1_ext_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    ext_sandwich_df, 'plus_1', aa_dict, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_plus1_ext_aa_pair_propensity_dict, upper_percentile_plus1_ext_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    plus1_ext_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    plus1_ext_aa_pair_propensity, lower_percentile_plus1_ext_aa_pair_propensity_dict,
    upper_percentile_plus1_ext_aa_pair_propensity_dict, 'Propensity', True, 'Ext_Plus1_pairs_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_plus1_ext_aa_pair_frequency_dict, upper_percentile_plus1_ext_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    plus1_ext_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    plus1_ext_aa_pair_frequency, lower_percentile_plus1_ext_aa_pair_frequency_dict,
    upper_percentile_plus1_ext_aa_pair_frequency_dict, 'Frequency', True, 'Ext_Plus1_pairs_frequency_95_conf'
)

#### Pairs in van der Waals contact

All sandwich surfaces

In [ ]:
vdw_aa_pairs = gen_neighbouring_pairs_list(sandwich_df, 'van_der_waals', aa_dict)
(vdw_aa_pair_propensity, vdw_aa_pair_frequency, vdw_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(vdw_aa_pairs, aa_dict)

In [ ]:
vdw_aa_pair_propensity

In [ ]:
vdw_aa_pair_frequency

In [ ]:
plot_heat_map(vdw_aa_pair_propensity, True, 'VDW_pairs_propensity')

In [ ]:
plot_heat_map(vdw_aa_pair_frequency, True, 'VDW_pairs_frequency')

In [ ]:
(
    vdw_aa_pair_bootstrap_propensity_dict, vdw_aa_pair_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    sandwich_df, 'van_der_waals', aa_dict, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    vdw_aa_pair_propensity, vdw_aa_pair_frequency
)

In [ ]:
(
    vdw_aa_pair_conf_intv_propensity_dict, vdw_aa_pair_conf_intv_frequency_dict,
    vdw_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    sandwich_df, 'van_der_waals', aa_dict, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_vdw_aa_pair_propensity_dict, upper_percentile_vdw_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    vdw_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    vdw_aa_pair_propensity, lower_percentile_vdw_aa_pair_propensity_dict,
    upper_percentile_vdw_aa_pair_propensity_dict, 'Propensity', True, 'VDW_pairs_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_vdw_aa_pair_frequency_dict, upper_percentile_vdw_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    vdw_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    vdw_aa_pair_frequency, lower_percentile_vdw_aa_pair_frequency_dict,
    upper_percentile_vdw_aa_pair_frequency_dict, 'Frequency', True, 'VDW_pairs_frequency_95_conf'
)

Interior surface

In [ ]:
vdw_int_aa_pairs = gen_neighbouring_pairs_list(int_sandwich_df, 'van_der_waals', aa_dict)
(vdw_int_aa_pair_propensity, vdw_int_aa_pair_frequency, vdw_int_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(vdw_int_aa_pairs, aa_dict)

In [ ]:
vdw_int_aa_pair_propensity

In [ ]:
vdw_int_aa_pair_frequency

In [ ]:
plot_heat_map(vdw_int_aa_pair_propensity, True, 'Int_VDW_pairs_propensity')

In [ ]:
plot_heat_map(vdw_int_aa_pair_frequency, True, 'Int_VDW_pairs_frequency')

In [ ]:
(
    vdw_int_aa_pair_bootstrap_propensity_dict, vdw_int_aa_pair_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    int_sandwich_df, 'van_der_waals', aa_dict, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    vdw_int_aa_pair_propensity, vdw_int_aa_pair_frequency
)

In [ ]:
(
    vdw_int_aa_pair_conf_intv_propensity_dict, vdw_int_aa_pair_conf_intv_frequency_dict,
    vdw_int_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    int_sandwich_df, 'van_der_waals', aa_dict, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_vdw_int_aa_pair_propensity_dict, upper_percentile_vdw_int_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    vdw_int_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    vdw_int_aa_pair_propensity, lower_percentile_vdw_int_aa_pair_propensity_dict,
    upper_percentile_vdw_int_aa_pair_propensity_dict, 'Propensity', True, 'Int_VDW_pairs_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_vdw_int_aa_pair_frequency_dict, upper_percentile_vdw_int_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    vdw_int_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    vdw_int_aa_pair_frequency, lower_percentile_vdw_int_aa_pair_frequency_dict,
    upper_percentile_vdw_int_aa_pair_frequency_dict, 'Frequency', True, 'Int_VDW_pairs_frequency_95_conf'
)

Exterior surface

In [ ]:
vdw_ext_aa_pairs = gen_neighbouring_pairs_list(ext_sandwich_df, 'van_der_waals', aa_dict)
(vdw_ext_aa_pair_propensity, vdw_ext_aa_pair_frequency, vdw_ext_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(vdw_ext_aa_pairs, aa_dict)

In [ ]:
vdw_ext_aa_pair_propensity

In [ ]:
vdw_ext_aa_pair_frequency

In [ ]:
plot_heat_map(vdw_ext_aa_pair_propensity, True, 'Ext_VDW_pairs_propensity')

In [ ]:
plot_heat_map(plus2_ext_aa_pair_frequency, True, 'Ext_VDW_pairs_frequency')

In [ ]:
(
    vdw_ext_aa_pair_bootstrap_propensity_dict, vdw_ext_aa_pair_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    ext_sandwich_df, 'van_der_waals', aa_dict, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    vdw_ext_aa_pair_propensity, vdw_ext_aa_pair_frequency
)

In [ ]:
(
    vdw_ext_aa_pair_conf_intv_propensity_dict, vdw_ext_aa_pair_conf_intv_frequency_dict,
    vdw_ext_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    ext_sandwich_df, 'van_der_waals', aa_dict, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_vdw_ext_aa_pair_propensity_dict, upper_percentile_vdw_ext_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    vdw_ext_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    vdw_ext_aa_pair_propensity, lower_percentile_vdw_ext_aa_pair_propensity_dict,
    upper_percentile_vdw_ext_aa_pair_propensity_dict, 'Propensity', True, 'Ext_VDW_pairs_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_vdw_ext_aa_pair_frequency_dict, upper_percentile_vdw_ext_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    vdw_ext_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    vdw_ext_aa_pair_frequency, lower_percentile_vdw_ext_aa_pair_frequency_dict,
    upper_percentile_vdw_ext_aa_pair_frequency_dict, 'Frequency', True, 'Ext_VDW_pairs_frequency_95_conf'
)

Interior/central

In [ ]:
vdw_cen_int_aa_pairs = gen_neighbouring_pairs_list(cen_int_sandwich_df, 'van_der_waals', aa_dict)
(vdw_cen_int_aa_pair_propensity, vdw_cen_int_aa_pair_frequency, vdw_cen_int_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(vdw_cen_int_aa_pairs, aa_dict)

In [ ]:
vdw_cen_int_aa_pair_propensity

In [ ]:
vdw_cen_int_aa_pair_frequency

In [ ]:
plot_heat_map(vdw_cen_int_aa_pair_propensity, True, 'Int_cen_VDW_pairs_propensity')

In [ ]:
plot_heat_map(vdw_cen_int_aa_pair_frequency, True, 'Int_cen_VDW_pairs_frequency')

In [ ]:
(
    vdw_cen_int_aa_pair_bootstrap_propensity_dict, vdw_cen_int_aa_pair_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    cen_int_sandwich_df, 'van_der_waals', aa_dict, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    vdw_cen_int_aa_pair_propensity, vdw_cen_int_aa_pair_frequency
)

In [ ]:
(
    vdw_cen_int_aa_pair_conf_intv_propensity_dict, vdw_cen_int_aa_pair_conf_intv_frequency_dict,
    vdw_cen_int_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    cen_int_sandwich_df, 'van_der_waals', aa_dict, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_vdw_cen_int_aa_pair_propensity_dict, upper_percentile_vdw_cen_int_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    vdw_cen_int_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    vdw_cen_int_aa_pair_propensity, lower_percentile_vdw_cen_int_aa_pair_propensity_dict,
    upper_percentile_vdw_cen_int_aa_pair_propensity_dict, 'Propensity', True, 'Int_cen_VDW_pairs_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_vdw_cen_int_aa_pair_frequency_dict, upper_percentile_vdw_cen_int_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    vdw_cen_int_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    vdw_cen_int_aa_pair_frequency, lower_percentile_vdw_cen_int_aa_pair_frequency_dict,
    upper_percentile_vdw_cen_int_aa_pair_frequency_dict, 'Frequency', True, 'Int_cen_VDW_pairs_frequency_95_conf'
)

Interior/edge

In [ ]:
vdw_edg_int_aa_pairs = gen_neighbouring_pairs_list(edg_int_sandwich_df, 'van_der_waals', aa_dict)
(vdw_edg_int_aa_pair_propensity, vdw_edg_int_aa_pair_frequency, vdw_edg_int_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(vdw_edg_int_aa_pairs, aa_dict)

In [ ]:
vdw_edg_int_aa_pair_propensity

In [ ]:
vdw_edg_int_aa_pair_frequency

In [ ]:
plot_heat_map(vdw_edg_int_aa_pair_propensity, True, 'Int_edg_VDW_pairs_propensity')

In [ ]:
plot_heat_map(vdw_edg_int_aa_pair_frequency, True, 'Int_edg_VDW_pairs_frequency')

In [ ]:
(
    vdw_edg_int_aa_pair_bootstrap_propensity_dict, vdw_edg_int_aa_pair_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    edg_int_sandwich_df, 'van_der_waals', aa_dict, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    vdw_edg_int_aa_pair_propensity, vdw_edg_int_aa_pair_frequency
)

In [ ]:
(
    vdw_edg_int_aa_pair_conf_intv_propensity_dict, vdw_edg_int_aa_pair_conf_intv_frequency_dict,
    vdw_edg_int_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    edg_int_sandwich_df, 'van_der_waals', aa_dict, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_vdw_edg_int_aa_pair_propensity_dict, upper_percentile_vdw_edg_int_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    vdw_edg_int_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    vdw_edg_int_aa_pair_propensity, lower_percentile_vdw_edg_int_aa_pair_propensity_dict,
    upper_percentile_vdw_edg_int_aa_pair_propensity_dict, 'Propensity', True, 'Int_edg_VDW_pairs_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_vdw_edg_int_aa_pair_frequency_dict, upper_percentile_vdw_edg_int_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    vdw_edg_int_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    vdw_edg_int_aa_pair_frequency, lower_percentile_vdw_edg_int_aa_pair_frequency_dict,
    upper_percentile_vdw_edg_int_aa_pair_frequency_dict, 'Frequency', True, 'Int_edg_VDW_pairs_frequency_95_conf'
)

Exterior/central

In [ ]:
vdw_cen_ext_aa_pairs = gen_neighbouring_pairs_list(cen_ext_sandwich_df, 'van_der_waals', aa_dict)
(vdw_cen_ext_aa_pair_propensity, vdw_cen_ext_aa_pair_frequency, vdw_cen_ext_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(vdw_cen_ext_aa_pairs, aa_dict)

In [ ]:
vdw_cen_ext_aa_pair_propensity

In [ ]:
vdw_cen_ext_aa_pair_frequency

In [ ]:
plot_heat_map(vdw_cen_ext_aa_pair_propensity, True, 'Ext_cen_VDW_pairs_propensity')

In [ ]:
plot_heat_map(vdw_cen_ext_aa_pair_frequency, True, 'Ext_cen_VDW_pairs_frequency')

In [ ]:
(
    vdw_cen_ext_aa_pair_bootstrap_propensity_dict, vdw_cen_ext_aa_pair_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    cen_ext_sandwich_df, 'van_der_waals', aa_dict, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    vdw_cen_ext_aa_pair_propensity, vdw_cen_ext_aa_pair_frequency
)

In [ ]:
(
    vdw_cen_ext_aa_pair_conf_intv_propensity_dict, vdw_cen_ext_aa_pair_conf_intv_frequency_dict,
    vdw_cen_ext_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    cen_ext_sandwich_df, 'van_der_waals', aa_dict, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_vdw_cen_ext_aa_pair_propensity_dict, upper_percentile_vdw_cen_ext_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    vdw_cen_ext_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    vdw_cen_ext_aa_pair_propensity, lower_percentile_vdw_cen_ext_aa_pair_propensity_dict,
    upper_percentile_vdw_cen_ext_aa_pair_propensity_dict, 'Propensity', True, 'Ext_cen_VDW_pairs_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_vdw_cen_ext_aa_pair_frequency_dict, upper_percentile_vdw_cen_ext_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    vdw_cen_ext_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    vdw_cen_ext_aa_pair_frequency, lower_percentile_vdw_cen_ext_aa_pair_frequency_dict,
    upper_percentile_vdw_cen_ext_aa_pair_frequency_dict, 'Frequency', True, 'Ext_cen_VDW_pairs_frequency_95_conf'
)

Exterior/edge

In [ ]:
vdw_edg_ext_aa_pairs = gen_neighbouring_pairs_list(edg_ext_sandwich_df, 'van_der_waals', aa_dict)
(vdw_edg_ext_aa_pair_propensity, vdw_edg_ext_aa_pair_frequency, vdw_edg_ext_aa_pair_normed_frequencies
) = calc_aa_pair_propensities(vdw_edg_ext_aa_pairs, aa_dict)

In [ ]:
vdw_edg_ext_aa_pair_propensity

In [ ]:
vdw_edg_ext_aa_pair_frequency

In [ ]:
plot_heat_map(vdw_edg_ext_aa_pair_propensity, True, 'Ext_edg_VDW_pairs_propensity')

In [ ]:
plot_heat_map(vdw_edg_ext_aa_pair_frequency, True, 'Ext_edg_VDW_pairs_frequency')

In [ ]:
(
    vdw_edg_ext_aa_pair_bootstrap_propensity_dict, vdw_edg_ext_aa_pair_bootstrap_frequency_dict
) = iterate_bootstrap_propensities(
    edg_ext_sandwich_df, 'van_der_waals', aa_dict, [10, 30, 100, 300, 1000], calc_aa_pair_propensities,
    vdw_edg_ext_aa_pair_propensity, vdw_edg_ext_aa_pair_frequency
)

In [ ]:
(
    vdw_edg_ext_aa_pair_conf_intv_propensity_dict, vdw_edg_ext_aa_pair_conf_intv_frequency_dict,
    vdw_edg_ext_aa_pair_conf_intv_normed_frequencies_dict
) = bootstrap_discrete_propensities(
    edg_ext_sandwich_df, 'van_der_waals', aa_dict, 300, calc_aa_pair_propensities
)

In [ ]:
(
    lower_percentile_vdw_edg_ext_aa_pair_propensity_dict, upper_percentile_vdw_edg_ext_aa_pair_propensity_dict
) = gen_95_conf_intervals(
    vdw_edg_ext_aa_pair_conf_intv_propensity_dict, 'Propensity'
)
plot_bar_graphs_with_conf_limits(
    vdw_edg_ext_aa_pair_propensity, lower_percentile_vdw_edg_ext_aa_pair_propensity_dict,
    upper_percentile_vdw_edg_ext_aa_pair_propensity_dict, 'Propensity', True, 'Ext_edg_VDW_pairs_propensity_95_conf'
)

In [ ]:
(
    lower_percentile_vdw_edg_ext_aa_pair_frequency_dict, upper_percentile_vdw_edg_ext_aa_pair_frequency_dict
) = gen_95_conf_intervals(
    vdw_edg_ext_aa_pair_conf_intv_frequency_dict, 'Frequency'
)
plot_bar_graphs_with_conf_limits(
    vdw_edg_ext_aa_pair_frequency, lower_percentile_vdw_edg_ext_aa_pair_frequency_dict,
    upper_percentile_vdw_edg_ext_aa_pair_frequency_dict, 'Frequency', True, 'Ext_edg_VDW_pairs_frequency_95_conf'
)

## Continuous propensity calculations

In [ ]:
os.chdir('..')
if not os.path.isdir('Continuous_propensities_indv_aas'):
    os.mkdir('Continuous_propensities_indv_aas')
os.chdir('Continuous_propensities_indv_aas')

### 1D individual amino acid propensities

Both sandwich surfaces

In [ ]:
plot_aa_kdes(sandwich_df, ['sandwich_z_coords'], 'comparison', aa_dict, True, 'sandwich_z_coord_dist')

In [ ]:
plot_bootstrap_aa_kdes(sandwich_df, 'sandwich_z_coords', aa_dict, 100, True, 'sandwich_z_coord_dist_100_bootstrap')

In [ ]:
z_sandwich_propensity = plot_1d_indv_aa_propensities(
    sandwich_df, 'sandwich_z_coords', aa_dict, 'True', 'sandwich_z_coord_propensity'
)

In [ ]:
plot_aa_kdes(sandwich_df, ['strand_z_coords'], 'comparison', aa_dict, True, 'strand_z_coord_dist')

In [ ]:
plot_bootstrap_aa_kdes(sandwich_df, 'strand_z_coords', aa_dict, 100, True, 'strand_z_coord_dist_100_bootstrap')

In [ ]:
z_strand_propensity = plot_1d_indv_aa_propensities(
    sandwich_df, 'strand_z_coords', aa_dict, True, 'strand_z_coord_propensity'
)

Interior surface

In [ ]:
plot_aa_kdes(int_sandwich_df, ['sandwich_z_coords'], 'comparison', aa_dict, True, 'Int_sandwich_z_coord_dist')

In [ ]:
plot_bootstrap_aa_kdes(
    int_sandwich_df, 'sandwich_z_coords', aa_dict, 100, True, 'Int_sandwich_z_coord_dist_100_bootstrap'
)

In [ ]:
z_sandwich_int_propensity = plot_1d_indv_aa_propensities(
    int_sandwich_df, 'sandwich_z_coords', aa_dict, True, 'Int_sandwich_z_coord_propensity'
)

In [ ]:
plot_aa_kdes(int_sandwich_df, ['strand_z_coords'], 'comparison', aa_dict, True, 'Int_strand_z_coord_dist')

In [ ]:
plot_bootstrap_aa_kdes(
    int_sandwich_df, 'strand_z_coords', aa_dict, 100, True, 'Int_strand_z_coord_dist_100_bootstrap'
)

In [ ]:
z_strand_int_propensity = plot_1d_indv_aa_propensities(
    int_sandwich_df, 'strand_z_coords', aa_dict, True, 'Int_strand_z_coord_propensity'
)

Exterior surface

In [ ]:
plot_aa_kdes(ext_sandwich_df, ['sandwich_z_coords'], 'comparison', aa_dict, True, 'Ext_sandwich_z_coord_dist')

In [ ]:
plot_bootstrap_aa_kdes(
    ext_sandwich_df, 'sandwich_z_coords', aa_dict, 100, True, 'Ext_sandwich_z_coord_dist_100_bootstrap'
)

In [ ]:
z_sandwich_ext_propensity = plot_1d_indv_aa_propensities(
    ext_sandwich_df, 'sandwich_z_coords', aa_dict, True, 'Ext_sandwich_z_coord_propensity'
)

In [ ]:
plot_aa_kdes(ext_sandwich_df, ['strand_z_coords'], 'comparison', aa_dict, True, 'Ext_strand_z_coord_dist')

In [ ]:
plot_bootstrap_aa_kdes(
    ext_sandwich_df, 'strand_z_coords', aa_dict, 100, True, 'Ext_strand_z_coord_dist_100_bootstrap'
)

In [ ]:
z_strand_ext_propensity = plot_1d_indv_aa_propensities(
    ext_sandwich_df, 'strand_z_coords', aa_dict, True, 'Ext_strand_z_coord_propensity'
)

### 1D pairwise amino acid propensities

In [ ]:
os.chdir('..')
if not os.path.isdir('Continuous_propensities_pairs_aas'):
    os.mkdir('Continuous_propensities_pairs_aas')
os.chdir('Continuous_propensities_pairs_aas')

#### Hydrogen bonding pairs

Both sandwich surfaces

In [ ]:
z_sandwich_hb_propensity = plot_1d_pairwise_aa_propensities(
    sandwich_df, 'hb_pairs', 'sandwich_z_coords', aa_dict, True, 'HB_sandwich_z_coords'
)

In [ ]:
z_strand_hb_propensity = plot_1d_pairwise_aa_propensities(
    sandwich_df, 'hb_pairs', 'strand_z_coords', aa_dict, True, 'HB_strand_z_coords'
)

Interior sandwich surface

In [ ]:
z_sandwich_hb_int_propensity = plot_1d_pairwise_aa_propensities(
    int_sandwich_df, 'hb_pairs', 'sandwich_z_coords', aa_dict, True, 'Int_HB_sandwich_z_coords'
)

In [ ]:
z_strand_hb_int_propensity = plot_1d_pairwise_aa_propensities(
    int_sandwich_df, 'hb_pairs', 'strand_z_coords', aa_dict, True, 'Int_HB_strand_z_coords'
)

Exterior sandwich surface

In [ ]:
z_sandwich_hb_ext_propensity = plot_1d_pairwise_aa_propensities(
    ext_sandwich_df, 'hb_pairs', 'sandwich_z_coords', aa_dict, True, 'Ext_HB_sandwich_z_coords'
)

In [ ]:
z_strand_hb_ext_propensity = plot_1d_pairwise_aa_propensities(
    ext_sandwich_df, 'hb_pairs', 'strand_z_coords', aa_dict, True, 'Ext_HB_strand_z_coords'
)

#### Non-hydrogen bonding pairs

Both sandwich surfaces

In [ ]:
z_sandwich_nhb_propensity = plot_1d_pairwise_aa_propensities(
    sandwich_df, 'nhb_pairs', 'sandwich_z_coords', aa_dict, True, 'NHB_sandwich_z_coords'
)

In [ ]:
z_strand_nhb_propensity = plot_1d_pairwise_aa_propensities(
    sandwich_df, 'nhb_pairs', 'strand_z_coords', aa_dict, True, 'NHB_strand_z_coords'
)

Interior sandwich surface

In [ ]:
z_sandwich_nhb_int_propensity = plot_1d_pairwise_aa_propensities(
    int_sandwich_df, 'nhb_pairs', 'sandwich_z_coords', aa_dict, True, 'Int_NHB_sandwich_z_coords'
)

In [ ]:
z_strand_nhb_int_propensity = plot_1d_pairwise_aa_propensities(
    int_sandwich_df, 'nhb_pairs', 'strand_z_coords', aa_dict, True, 'Int_NHB_strand_z_coords'
)

Exterior sandwich surface

In [ ]:
z_sandwich_nhb_ext_propensity = plot_1d_pairwise_aa_propensities(
    ext_sandwich_df, 'nhb_pairs', 'sandwich_z_coords', aa_dict, True, 'Ext_NHB_sandwich_z_coords'
)

In [ ]:
z_strand_nhb_ext_propensity = plot_1d_pairwise_aa_propensities(
    ext_sandwich_df, 'nhb_pairs', 'strand_z_coords', aa_dict, True, 'Ext_NHB_strand_z_coords'
)

#### Minus 2 pairs

Both sandwich surfaces

In [ ]:
z_sandwich_minus2_propensity = plot_1d_pairwise_aa_propensities(
    sandwich_df, 'minus_2', 'sandwich_z_coords', aa_dict, True, 'Minus2_sandwich_z_coords'
)

In [ ]:
z_strand_minus2_propensity = plot_1d_pairwise_aa_propensities(
    sandwich_df, 'minus_2', 'strand_z_coords', aa_dict, True, 'Minus2_strand_z_coords'
)

Interior sandwich surface

In [ ]:
z_sandwich_minus2_int_propensity = plot_1d_pairwise_aa_propensities(
    int_sandwich_df, 'minus_2', 'sandwich_z_coords', aa_dict, True, 'Int_Minus2_sandwich_z_coords'
)

In [ ]:
z_strand_minus2_int_propensity = plot_1d_pairwise_aa_propensities(
    int_sandwich_df, 'minus_2', 'strand_z_coords', aa_dict, True, 'Int_Minus2_strand_z_coords'
)

Exterior sandwich surface

In [ ]:
z_sandwich_minus2_ext_propensity = plot_1d_pairwise_aa_propensities(
    ext_sandwich_df, 'minus_2', 'sandwich_z_coords', aa_dict, True, 'Ext_Minus2_sandwich_z_coords'
)

In [ ]:
z_strand_minus2_ext_propensity = plot_1d_pairwise_aa_propensities(
    ext_sandwich_df, 'minus_2', 'strand_z_coords', aa_dict, True, 'Ext_Minus2_strand_z_coords'
)

#### Plus 2 pairs

Both sandwich surfaces

In [ ]:
z_sandwich_plus2_propensity = plot_1d_pairwise_aa_propensities(
    sandwich_df, 'plus_2', 'sandwich_z_coords', aa_dict, True, 'Plus2_sandwich_z_coords'
)

In [ ]:
z_strand_plus2_propensity = plot_1d_pairwise_aa_propensities(
    sandwich_df, 'plus_2', 'strand_z_coords', aa_dict, True, 'Plus2_strand_z_coords'
)

Interior sandwich surface

In [ ]:
z_sandwich_plus2_int_propensity = plot_1d_pairwise_aa_propensities(
    int_sandwich_df, 'plus_2', 'sandwich_z_coords', aa_dict, True, 'Int_Plus2_sandwich_z_coords'
)

In [ ]:
z_strand_plus2_int_propensity = plot_1d_pairwise_aa_propensities(
    int_sandwich_df, 'plus_2', 'strand_z_coords', aa_dict, True, 'Int_Plus2_strand_z_coords'
)

Exterior sandwich surface

In [ ]:
z_sandwich_plus2_ext_propensity = plot_1d_pairwise_aa_propensities(
    ext_sandwich_df, 'plus_2', 'sandwich_z_coords', aa_dict, True, 'Ext_Plus2_sandwich_z_coords'
)

In [ ]:
z_strand_plus2_ext_propensity = plot_1d_pairwise_aa_propensities(
    ext_sandwich_df, 'plus_2', 'strand_z_coords', aa_dict, True, 'Ext_Plus2_strand_z_coords'
)

#### Minus 1 pairs

Both sandwich surfaces

In [ ]:
z_sandwich_minus1_propensity = plot_1d_pairwise_aa_propensities(
    sandwich_df, 'minus_1', 'sandwich_z_coords', aa_dict, True, 'Minus1_sandwich_z_coords'
)

In [ ]:
z_strand_minus1_propensity = plot_1d_pairwise_aa_propensities(
    sandwich_df, 'minus_1', 'strand_z_coords', aa_dict, True, 'Minus1_strand_z_coords'
)

Interior sandwich surface

In [ ]:
z_sandwich_minus1_int_propensity = plot_1d_pairwise_aa_propensities(
    int_sandwich_df, 'minus_1', 'sandwich_z_coords', aa_dict, True, 'Int_Minus1_sandwich_z_coords'
)

In [ ]:
z_strand_minus1_int_propensity = plot_1d_pairwise_aa_propensities(
    int_sandwich_df, 'minus_1', 'strand_z_coords', aa_dict, True, 'Int_Minus1_strand_z_coords'
)

Exterior sandwich surface

In [ ]:
z_sandwich_minus1_ext_propensity = plot_1d_pairwise_aa_propensities(
    ext_sandwich_df, 'minus_1', 'sandwich_z_coords', aa_dict, True, 'Ext_Minus1_sandwich_z_coords'
)

In [ ]:
z_strand_minus1_ext_propensity = plot_1d_pairwise_aa_propensities(
    ext_sandwich_df, 'minus_1', 'strand_z_coords', aa_dict, True, 'Ext_Minus1_strand_z_coords'
)

#### Plus 1 pairs

Both sandwich surfaces

In [ ]:
z_sandwich_plus1_propensity = plot_1d_pairwise_aa_propensities(
    sandwich_df, 'plus_1', 'sandwich_z_coords', aa_dict, True, 'Plus1_sandwich_z_coords'
)

In [ ]:
z_strand_plus1_propensity = plot_1d_pairwise_aa_propensities(
    sandwich_df, 'plus_1', 'strand_z_coords', aa_dict, True, 'Plus1_strand_z_coords'
)

Interior sandwich surface

In [ ]:
z_sandwich_plus1_int_propensity = plot_1d_pairwise_aa_propensities(
    int_sandwich_df, 'plus_1', 'sandwich_z_coords', aa_dict, True, 'Int_Plus1_sandwich_z_coords'
)

In [ ]:
z_strand_plus1_int_propensity = plot_1d_pairwise_aa_propensities(
    int_sandwich_df, 'plus_1', 'strand_z_coords', aa_dict, True, 'Int_Plus1_strand_z_coords'
)

Exterior sandwich surface

In [ ]:
z_sandwich_plus1_ext_propensity = plot_1d_pairwise_aa_propensities(
    ext_sandwich_df, 'plus_1', 'sandwich_z_coords', aa_dict, True, 'Ext_Plus1_sandwich_z_coords'
)

In [ ]:
z_strand_plus1_ext_propensity = plot_1d_pairwise_aa_propensities(
    ext_sandwich_df, 'plus_1', 'strand_z_coords', aa_dict, True, 'Ext_Plus1_strand_z_coords'
)

#### Van der Waals pairs

Both sandwich surfaces

In [ ]:
z_sandwich_vdw_propensity = plot_1d_pairwise_aa_propensities(
    sandwich_df, 'van_der_waals', 'sandwich_z_coords', aa_dict, True, 'VDW_sandwich_z_coords'
)

In [ ]:
z_strand_vdw_propensity = plot_1d_pairwise_aa_propensities(
    sandwich_df, 'van_der_waals', 'strand_z_coords', aa_dict, True, 'VDW_strand_z_coords'
)

Interior sandwich surface

In [ ]:
z_sandwich_vdw_int_propensity = plot_1d_pairwise_aa_propensities(
    int_sandwich_df, 'van_der_waals', 'sandwich_z_coords', aa_dict, True, 'Int_VDW_sandwich_z_coords'
)

In [ ]:
z_strand_vdw_int_propensity = plot_1d_pairwise_aa_propensities(
    int_sandwich_df, 'van_der_waals', 'strand_z_coords', aa_dict, True, 'Int_VDW_strand_z_coords'
)

Exterior sandwich surface

In [ ]:
z_sandwich_vdw_ext_propensity = plot_1d_pairwise_aa_propensities(
    ext_sandwich_df, 'van_der_waals', 'sandwich_z_coords', aa_dict, True, 'Ext_VDW_sandwich_z_coords'
)

In [ ]:
z_strand_vdw_ext_propensity = plot_1d_pairwise_aa_propensities(
    ext_sandwich_df, 'van_der_waals', 'strand_z_coords', aa_dict, True, 'Ext_VDW_strand_z_coords'
)

## Saves propensity dictionaries to pickle file

In [ ]:
os.chdir('../..')

In [ ]:
propensities_dict = OrderedDict({'int_-_-_-_indv_disc_propensity': int_ext_propensity.loc[:, ['FASTA', 'interior']],
                                 'ext_-_-_-_indv_disc_propensity': int_ext_propensity.loc[:, ['FASTA', 'exterior']],
                                 'int_edg_-_-_indv_disc_propensity': edg_cen_int_ext_propensity.loc[:, ['FASTA', 'edge_interior']],
                                 'ext_edg_-_-_indv_disc_propensity': edg_cen_int_ext_propensity.loc[:, ['FASTA', 'edge_exterior']],
                                 'int_cen_-_-_indv_disc_propensity': edg_cen_int_ext_propensity.loc[:, ['FASTA', 'central_interior']],
                                 'ext_cen_-_-_indv_disc_propensity': edg_cen_int_ext_propensity.loc[:, ['FASTA', 'central_exterior']],
                                 '-_-_-_hb_pair_disc_propensity': hb_aa_pair_propensity,
                                 'int_-_-_hb_pair_disc_propensity': hb_int_aa_pair_propensity,
                                 'ext_-_-_hb_pair_disc_propensity': hb_ext_aa_pair_propensity,
                                 '-_-_-_nhb_pair_disc_propensity': nhb_aa_pair_propensity,
                                 'int_-_-_nhb_pair_disc_propensity': nhb_int_aa_pair_propensity,
                                 'ext_-_-_nhb_pair_disc_propensity': nhb_ext_aa_pair_propensity,
                                 '-_-_-_plusminus2_pair_disc_propensity': minus2_aa_pair_propensity,
                                 'int_-_-_plusminus2_pair_disc_propensity': minus2_int_aa_pair_propensity,
                                 'ext_-_-_plusminus2_pair_disc_propensity': minus2_ext_aa_pair_propensity,
                                 '-_-_-_plusminus1_pair_disc_propensity': minus1_aa_pair_propensity,
                                 'int_-_-_plusminus1_pair_disc_propensity': minus1_int_aa_pair_propensity,
                                 'ext_-_-_plusminus1_pair_disc_propensity': minus1_ext_aa_pair_propensity,
                                 '-_-_-_vdw_pair_disc_propensity': vdw_aa_pair_propensity,
                                 'int_-_-_vdw_pair_disc_propensity': vdw_int_aa_pair_propensity,
                                 'ext_-_-_vdw_pair_disc_propensity': vdw_ext_aa_pair_propensity,
                                 '-_-_zsandwich_-_indv_cont_propensity': z_sandwich_propensity,
                                 'int_-_zsandwich_-_indv_cont_propensity': z_sandwich_int_propensity,
                                 'ext_-_zsandwich_-_indv_cont_propensity': z_sandwich_ext_propensity,
                                 '-_-_zstrand_-_indv_cont_propensity': z_strand_propensity,
                                 'int_-_zstrand_-_indv_cont_propensity': z_strand_int_propensity,
                                 'ext_-_zstrand_-_indv_cont_propensity': z_strand_ext_propensity,
                                 '-_-_phipsi_-_indv_disc_propensity': discrete_phi_psi_propensity,
                                 'int_-_phipsi_-_indv_disc_propensity': discrete_phi_psi_int_propensity,
                                 'ext_-_phipsi_-_indv_disc_propensity': discrete_phi_psi_ext_propensity})

with open('Pickled_sandwich_propensity_dictionaries.pkl', 'wb') as pickle_file:
    pickle.dump(propensities_dict, pickle_file)

In [ ]:
frequencies_dict = OrderedDict({'int_-_-_-_indv_disc_frequency': int_ext_normed_frequencies.loc[:, ['FASTA', 'interior']],
                                'ext_-_-_-_indv_disc_frequency': int_ext_normed_frequencies.loc[:, ['FASTA', 'exterior']],
                                'int_edg_-_-_indv_disc_frequency': edg_cen_int_ext_normed_frequencies.loc[:, ['FASTA', 'edge_interior']],
                                'ext_edg_-_-_indv_disc_frequency': edg_cen_int_ext_normed_frequencies.loc[:, ['FASTA', 'edge_exterior']],
                                'int_cen_-_-_indv_disc_frequency': edg_cen_int_ext_normed_frequencies.loc[:, ['FASTA', 'central_interior']],
                                'ext_cen_-_-_indv_disc_frequency': edg_cen_int_ext_normed_frequencies.loc[:, ['FASTA', 'central_exterior']],
                                '-_-_-_hb_pair_disc_frequency': hb_aa_pair_normed_frequencies,
                                'int_-_-_hb_pair_disc_frequency': hb_int_aa_pair_normed_frequencies,
                                'ext_-_-_hb_pair_disc_frequency': hb_ext_aa_pair_normed_frequencies,
                                '-_-_-_nhb_pair_disc_frequency': nhb_aa_pair_normed_frequencies,
                                'int_-_-_nhb_pair_disc_frequency': nhb_int_aa_pair_normed_frequencies,
                                'ext_-_-_nhb_pair_disc_frequency': nhb_ext_aa_pair_normed_frequencies,
                                '-_-_-_plusminus2_pair_disc_frequency': minus2_aa_pair_normed_frequencies,
                                'int_-_-_plusminus2_pair_disc_frequency': minus2_int_aa_pair_normed_frequencies,
                                'ext_-_-_plusminus2_pair_disc_frequency': minus2_ext_aa_pair_normed_frequencies,
                                '-_-_-_plusminus1_pair_disc_frequency': minus1_aa_pair_normed_frequencies,
                                'int_-_-_plusminus1_pair_disc_frequency': minus1_int_aa_pair_normed_frequencies,
                                'ext_-_-_plusminus1_pair_disc_frequency': minus1_ext_aa_pair_normed_frequencies,
                                '-_-_-_vdw_pair_disc_frequency': vdw_aa_pair_normed_frequencies,
                                'int_-_-_vdw_pair_disc_frequency': vdw_int_aa_pair_normed_frequencies,
                                'ext_-_-_vdw_pair_disc_frequency': vdw_ext_aa_pair_normed_frequencies,
                                '-_-_phipsi_-_indv_disc_frequency': discrete_phi_psi_normed_frequencies,
                                'int_-_phipsi_-_indv_disc_frequency': discrete_phi_psi_int_normed_frequencies,
                                'ext_-_phipsi_-_indv_disc_frequency': discrete_phi_psi_ext_normed_frequencies})

with open('Pickled_sandwich_frequency_dictionaries.pkl', 'wb') as pickle_file:
    pickle.dump(frequencies_dict, pickle_file)

In [ ]:
cluster_coords_dict = OrderedDict({'all_surfaces': discrete_phi_psi_cluster_coords,
                                   'interior': discrete_phi_psi_int_cluster_coords,
                                   'exterior': discrete_phi_psi_ext_cluster_coords})
with open('Pickled_phi_psi_sandwich_cluster_coords.pkl', 'wb') as pickle_file:
    pickle.dump((cluster_coords_dict), pickle_file)

## 2D Kolmogorov-Smirnov test to compare individual and overall amino acid z-coordinate distributions

In [ ]:
if not os.path.isdir('Sandwich_plots/KS_tests_for_1D_dist'):
    os.mkdir('Sandwich_plots/KS_tests_for_1D_dist')
os.chdir('Sandwich_plots/KS_tests_for_1D_dist')

Both sandwich surfaces

How many bootstrap samples should I use (all residues)?

In [ ]:
iterate_bootstrap_conf_limits(
    sandwich_df, 'sandwich_z_coords', run_ks_2samp_test, [100, 300, 1000, 3000, 10000], aa_dict
)

In [ ]:
iterate_bootstrap_conf_limits(
    sandwich_df, 'strand_z_coords', run_ks_2samp_test, [100, 300, 1000, 3000, 10000], aa_dict
)

Are the differences I observe between individual amino acid and overall amino acid distributions significant?

In [ ]:
initial_D_dict_z_sandwich, initial_p_dict_z_sandwich = initial_ks_test(
    sandwich_df, 'sandwich_z_coords', run_ks_2samp_test, aa_dict
)

D_dict_bootstrap_z_sandwich, p_dict_bootstrap_z_sandwich = bootstrap_ks_test(
    sandwich_df, 'sandwich_z_coords', 3000, run_ks_2samp_test, aa_dict
)

conf_intv_D_dict_z_sandwich = calc_95_conf_limits(initial_D_dict_z_sandwich, D_dict_bootstrap_z_sandwich)
draw_strip_plot(conf_intv_D_dict_z_sandwich, 'D', save_file=True, file_name='z_sandwich_KS_test_D_stat')

conf_intv_p_dict_z_sandwich = calc_95_conf_limits(initial_p_dict_z_sandwich, p_dict_bootstrap_z_sandwich)
draw_strip_plot(conf_intv_p_dict_z_sandwich, 'p', save_file=True, file_name='z_sandwich_KS_test_p_stat')

In [ ]:
initial_D_dict_z_strand, initial_p_dict_z_strand = initial_ks_test(
    sandwich_df, 'strand_z_coords', run_ks_2samp_test, aa_dict
)

D_dict_bootstrap_z_strand, p_dict_bootstrap_z_strand = bootstrap_ks_test(
    sandwich_df, 'strand_z_coords', 3000, run_ks_2samp_test, aa_dict
)

conf_intv_D_dict_z_strand = calc_95_conf_limits(initial_D_dict_z_strand, D_dict_bootstrap_z_strand)
draw_strip_plot(conf_intv_D_dict_z_strand, 'D', save_file=True, file_name='z_strand_KS_test_D_stat')

conf_intv_p_dict_z_strand = calc_95_conf_limits(initial_p_dict_z_strand, p_dict_bootstrap_z_strand)
draw_strip_plot(conf_intv_p_dict_z_strand, 'p', save_file=True, file_name='z_strand_KS_test_p_stat')

Interior surface

How many bootstrap samples should I use (all residues)?

In [ ]:
iterate_bootstrap_conf_limits(
    int_sandwich_df, 'sandwich_z_coords', run_ks_2samp_test, [100, 300, 1000, 3000, 10000], aa_dict
)

In [ ]:
iterate_bootstrap_conf_limits(
    int_sandwich_df, 'strand_z_coords', run_ks_2samp_test, [100, 300, 1000, 3000, 10000], aa_dict
)

Are the differences I observe between individual amino acid and overall amino acid distributions significant?

In [ ]:
initial_D_dict_z_sandwich_int, initial_p_dict_z_sandwich_int = initial_ks_test(
    int_sandwich_df, 'sandwich_z_coords', run_ks_2samp_test, aa_dict
)

D_dict_bootstrap_z_sandwich_int, p_dict_bootstrap_z_sandwich_int = bootstrap_ks_test(
    int_sandwich_df, 'sandwich_z_coords', 10, run_ks_2samp_test, aa_dict
)

conf_intv_D_dict_z_sandwich_int = calc_95_conf_limits(initial_D_dict_z_sandwich_int, D_dict_bootstrap_z_sandwich_int)
draw_strip_plot(conf_intv_D_dict_z_sandwich_int, 'D', save_file=True, file_name='z_sandwich_int_KS_test_D_stat')

conf_intv_p_dict_z_sandwich_int = calc_95_conf_limits(initial_p_dict_z_sandwich_int, p_dict_bootstrap_z_sandwich_int)
draw_strip_plot(conf_intv_p_dict_z_sandwich_int, 'p', save_file=True, file_name='z_sandwich_int_KS_test_p_stat')

In [ ]:
initial_D_dict_z_strand_int, initial_p_dict_z_strand_int = initial_ks_test(
    int_sandwich_df, 'strand_z_coords', run_ks_2samp_test, aa_dict
)

D_dict_bootstrap_z_strand_int, p_dict_bootstrap_z_strand_int = bootstrap_ks_test(
    int_sandwich_df, 'strand_z_coords', 10, run_ks_2samp_test, aa_dict
)

conf_intv_D_dict_z_strand_int = calc_95_conf_limits(initial_D_dict_z_strand_int, D_dict_bootstrap_z_strand_int)
draw_strip_plot(conf_intv_D_dict_z_strand_int, 'D', save_file=True, file_name='z_strand_int_KS_test_D_stat')

conf_intv_p_dict_z_strand_int = calc_95_conf_limits(initial_p_dict_z_strand_int, p_dict_bootstrap_z_strand_int)
draw_strip_plot(conf_intv_p_dict_z_strand_int, 'p', save_file=True, file_name='z_strand_int_KS_test_p_stat')

Exterior surface

How many bootstrap samples should I use (all residues)?

In [ ]:
iterate_bootstrap_conf_limits(
    ext_sandwich_df, 'sandwich_z_coords', run_ks_2samp_test, [100, 300, 1000, 3000, 10000, 30000, 100000], aa_dict
)

In [ ]:
iterate_bootstrap_conf_limits(
    ext_sandwich_df, 'strand_z_coords', run_ks_2samp_test, [100, 300, 1000, 3000, 10000, 30000, 100000], aa_dict
)

Are the differences I observe between individual amino acid and overall amino acid distributions significant?

In [ ]:
initial_D_dict_z_sandwich_ext, initial_p_dict_z_sandwich_ext = initial_ks_test(
    ext_sandwich_df, 'sandwich_z_coords', run_ks_2samp_test, aa_dict
)

D_dict_bootstrap_z_sandwich_ext, p_dict_bootstrap_z_sandwich_ext = bootstrap_ks_test(
    ext_sandwich_df, 'sandwich_z_coords', 10, run_ks_2samp_test, aa_dict
)

conf_intv_D_dict_z_sandwich_ext = calc_95_conf_limits(initial_D_dict_z_sandwich_ext, D_dict_bootstrap_z_sandwich_ext)
draw_strip_plot(conf_intv_D_dict_z_sandwich_ext, 'D', save_file=True, file_name='z_sandwich_ext_KS_test_D_stat')

conf_intv_p_dict_z_sandwich_ext = calc_95_conf_limits(initial_p_dict_z_sandwich_ext, p_dict_bootstrap_z_sandwich_ext)
draw_strip_plot(conf_intv_p_dict_z_sandwich_ext, 'p', save_file=True, file_name='z_sandwich_ext_KS_test_p_stat')

In [ ]:
initial_D_dict_z_strand_ext, initial_p_dict_z_strand_ext = initial_ks_test(
    ext_sandwich_df, 'strand_z_coords', run_ks_2samp_test, aa_dict
)

D_dict_bootstrap_z_strand_ext, p_dict_bootstrap_z_strand_ext = bootstrap_ks_test(
    ext_sandwich_df, 'strand_z_coords', 10, run_ks_2samp_test, aa_dict
)

conf_intv_D_dict_z_strand_ext = calc_95_conf_limits(initial_D_dict_z_strand_ext, D_dict_bootstrap_z_strand_ext)
draw_strip_plot(conf_intv_D_dict_z_strand_ext, 'D', save_file=True, file_name='z_strand_ext_KS_test_D_stat')

conf_intv_p_dict_z_strand_ext = calc_95_conf_limits(initial_p_dict_z_strand_ext, p_dict_bootstrap_z_strand_ext)
draw_strip_plot(conf_intv_p_dict_z_strand_ext, 'p', save_file=True, file_name='z_strand_ext_KS_test_p_stat')

## Interaction type vs. z-coordinate KDEs

In [ ]:
os.chdir('..')
if not os.path.isdir('Interaction_types_vs_z'):
    os.mkdir('Interaction_types_vs_z')
os.chdir('Interaction_types_vs_z')

In [ ]:
plot_discrete_prop_frequency_kdes(
    sandwich_df, 'sandwich_z_coords', ['van_der_waals', 'h_bonds', 'ionic', 'ss_bonds', 'pi_pi_stacking',
                                       'pi_pi_stacking_p', 'pi_pi_stacking_l', 'pi_pi_stacking_n', 'pi_pi_stacking_t',
                                       'cation_pi'],
    True, 'vs_z_sandwich'
)

In [ ]:
plot_discrete_prop_frequency_kdes(
    int_sandwich_df, 'sandwich_z_coords', ['van_der_waals', 'h_bonds', 'ionic', 'ss_bonds', 'pi_pi_stacking',
                                           'pi_pi_stacking_p', 'pi_pi_stacking_l', 'pi_pi_stacking_n',
                                           'pi_pi_stacking_t', 'cation_pi'],
    True, 'vs_z_sandwich_int'
)

In [ ]:
plot_discrete_prop_frequency_kdes(
    ext_sandwich_df, 'sandwich_z_coords', ['van_der_waals', 'h_bonds', 'ionic', 'ss_bonds', 'pi_pi_stacking',
                                           'pi_pi_stacking_p', 'pi_pi_stacking_l', 'pi_pi_stacking_n',
                                           'pi_pi_stacking_t', 'cation_pi'],
    True, 'vs_z_sandwich_ext'
)

In [ ]:
plot_discrete_prop_frequency_kdes(
    sandwich_df, 'strand_z_coords', ['van_der_waals', 'h_bonds', 'ionic', 'ss_bonds', 'pi_pi_stacking',
                                     'pi_pi_stacking_p', 'pi_pi_stacking_l', 'pi_pi_stacking_n', 'pi_pi_stacking_t',
                                     'cation_pi'],
    True, 'vs_z_strand'
)

In [ ]:
plot_discrete_prop_frequency_kdes(
    int_sandwich_df, 'strand_z_coords', ['van_der_waals', 'h_bonds', 'ionic', 'ss_bonds', 'pi_pi_stacking',
                                         'pi_pi_stacking_p', 'pi_pi_stacking_l', 'pi_pi_stacking_n',
                                         'pi_pi_stacking_t', 'cation_pi'],
    True, 'vs_z_strand_int'
)

In [ ]:
plot_discrete_prop_frequency_kdes(
    ext_sandwich_df, 'strand_z_coords', ['van_der_waals', 'h_bonds', 'ionic', 'ss_bonds', 'pi_pi_stacking',
                                         'pi_pi_stacking_p', 'pi_pi_stacking_l', 'pi_pi_stacking_n',
                                         'pi_pi_stacking_t', 'cation_pi'],
    True, 'vs_z_sandwich_ext'
)

### Standard error of proportion

In [ ]:
os.chdir('..')
if not os.path.isdir('Std_error_proportion_calcs'):
    os.mkdir('Std_error_proportion_calcs')
os.chdir('Std_error_proportion_calcs')

Recalculate HB / NHB standard error of proportion z-scores from count data in Dek's 1998 paper

In [ ]:
dek_sandwich_hb_pairs_dict, dek_sandwich_hb_pairs_total = gen_dek_sandwich_dicts(aa_dict, 'hb')
dek_sandwich_nhb_pairs_dict, dek_sandwich_nhb_pairs_total = gen_dek_sandwich_dicts(aa_dict, 'nhb')

dek_sandwich_hb_nhb_z_scores, dek_sandwich_hb_nhb_ratios = calc_std_error_proportion(
    aa_dict, dek_sandwich_hb_pairs_dict, dek_sandwich_nhb_pairs_dict, dek_sandwich_hb_pairs_total,
    dek_sandwich_nhb_pairs_total
)

In [ ]:
dek_sandwich_hb_nhb_ratios

In [ ]:
print_significant_z_scores(dek_sandwich_hb_nhb_z_scores)

Calculate HB / NHB standard error of proportion z-scores for new data

In [ ]:
sandwich_df_hb_pairs_count_dict, sandwich_df_hb_pairs_total = calc_pair_proportions(
    sandwich_df, 'hb_pairs', aa_dict
)
sandwich_df_nhb_pairs_count_dict, sandwich_df_nhb_pairs_total = calc_pair_proportions(
    sandwich_df, 'nhb_pairs', aa_dict
)

sandwich_df_hb_nhb_z_scores, sandwich_df_hb_nhb_ratios = calc_std_error_proportion(
    aa_dict, sandwich_df_hb_pairs_count_dict, sandwich_df_nhb_pairs_count_dict, sandwich_df_hb_pairs_total,
    sandwich_df_nhb_pairs_total
)

In [ ]:
sandwich_df_hb_nhb_ratios

In [ ]:
print_significant_z_scores(sandwich_df_hb_nhb_z_scores)

In [ ]:
sandwich_df_dek_sandwich_z_diff_dict = calc_z_diff(sandwich_df_hb_nhb_z_scores, dek_sandwich_hb_nhb_z_scores)

In [ ]:
sandwich_df_dek_sandwich_z_diff_dict

In [ ]:
with open('Pickled_HB_NHB_std_error_proportion_dicts.pkl', 'wb') as pickle_file:
    pickle.dump(
        (dek_sandwich_hb_nhb_z_scores, sandwich_df_hb_nhb_z_scores, sandwich_df_dek_sandwich_z_diff_dict),
        pickle_file
    )